In [10]:
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import json
import re
from tqdm import tqdm

In [71]:
def script_to_json(script_str):
    lines = script_str.strip('\n| |;').replace(';', ',').replace("'", '"').split('\n')
    processed_lines = []
    for line in lines:
        if line.find('=') < 0:
            continue
        splited = [i.strip(' |;') for i in line.split('=')]
        if splited[0][0] != '"':
            splited[0] = '"'+ splited[0] +'"'
        if splited[1][0] != '"' and splited[1][0] != '{':
            splited[1] = '"'+ splited[1].strip(',') +'",'
        processed_lines.append(': '.join(splited))
        
    processed_lines[-1] = processed_lines[-1].strip(',')
    
    try:
        json_str = '{\n' + '\n'.join(processed_lines) + '\n}'
        json_obj = json.loads(json_str.encode('utf-8'))
        return json_obj
    
    except Exception as e:
        print(e)
        print(json_str)
        return

In [257]:
url = "https://www.aminer.org/conf/ijcai2020/papers"

#If there is no such folder, the script will create one automatically
folder_location = r'D:\Valentina\Thesis\papers'
#if not os.path.exists(folder_location):os.mkdir(folder_location)

response = requests.get(url)

soup= BeautifulSoup(response.text, "html.parser") 

In [258]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="">
 <head>
  <title data-react-helmet="true">
   IJCAI2020-学术会议 - AMiner
  </title>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <!-- <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, minimum-scale=1, user-scalable=no" /> -->
  <link href="//ssr.aminer.cn/public/umi.605d5d22.css" rel="stylesheet"/>
  <link data-react-helmet="true" href="https://originalfileserver.aminer.cn/sys/aminer/favicon.ico" rel="icon" type="image/x-icon"/>
  <link data-react-helmet="true" href="https://originalfileserver.aminer.cn/lib/fa/css/font-awesome.min.css" rel="stylesheet"/>
  <script>
   window.routerBase = '/';
    window.resourceBaseUrl = '/public/';
  </script>
  <script src="https://static.aminer.cn/misc/ga.js">
  </script>
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-1156684-2">
  </script>
  <

In [218]:
for script in scripts:
    if script.string and script.string.find('window.g_initialProps') > -1:
        content = script.string.split('\n')[2].split(' = ')[1].strip(' |;').replace("'", "")
        content = content.replace('\\', '\\').replace('undefined', '"undefined"')

In [235]:
data_dict = json.loads(content)

In [259]:
data_dict

{'aminerConf': {'confInfo': {'address': 'Yokohama, Japan',
   'begin_date': '2021-01-05T10:11:12.818Z',
   'config': {'breadcrumb': True,
    'navigator': ['homepage',
     'papers',
     'roster',
     'news',
     'videos',
     'contact',
     'recommend'],
    'homepage': {'ads': True},
    'left': ['keywords', 'leftAuthors'],
    'right': ['statistics', 'insight'],
    'paper': {'navigator': ['paper'],
     'category': [],
     'download_org': [{'key': 'all',
       'val': '下载全部',
       'link': 'https://pan.baidu.com/s/11FioZCMwufp-hqLJHpzVzw ',
       'code': '9pls'}]},
    'roster': {'navigator': ['chinese-scholar', 'chinese-first', 'authors']},
    'tdk': {'pageTitle': 'IJCAI2020-学术会议',
     'pageDesc': 'IJCAI 2020 会议助手涵盖会议论文、作者列表、论文解读和论文推荐，提供全方位会议信息及分析。',
     'pageKeywords': 'IJCAI2020, ijcai, ijcai学术会议, Artificial Intelligence, 人工智能'}},
   'created_time': '2020-06-28T10:13:09.291Z',
   'creator': '54f5112e45ce1bc6d563b8d9',
   'eid': ['5efc4c4a4c775ed682ed406a', '5efd3b724c

In [245]:
papers = data_dict['aminerConf']['SearchPubsData']['items']

## Scraping a paper for example

In [247]:
papers[0]

{'abstract': 'Recent efforts on training visual navigation agents conditioned on language using deep reinforcement learning have been successful in learning policies for different multimodal tasks, such as semantic goal navigation and embodied question answering. In this paper, we propose a multitask model capable of jointly learning these multimodal tasks, and transferring knowledge of words and their grounding in visual objects across the tasks. The proposed model uses a novel Dual-Attention unit to disentangle the knowledge of words in the textual representations and visual concepts in the visual representations, and align them with each other. This disentangled task-invariant alignment of representations facilitates grounding and knowledge transfer across both tasks. We show that the proposed model outperforms a range of baselines on both tasks in simulated 3D environments. We also show that this disentanglement of representations makes our model modular, interpretable, and allows 

In [248]:
papers[0]['title']

'Embodied Multimodal Multitask Learning.'

In [249]:
papers[0]['pdf']

'https://static.aminer.cn/upload/pdf/program/5cede0f0da562983788cec6c_0.pdf'

In [250]:
folder_location = r'D:\Valentina\Thesis\papers'
filename = os.path.join(folder_location, papers[0]['title'] + '.pdf')
with open(filename, 'wb') as f:
    f.write(requests.get(papers[0]['pdf']).content)


## Scraping all papers an the page

In [254]:
folder_location = r'D:\Valentina\Thesis\papers'
for paper in tqdm(papers):
    title = paper['title'].replace('\n', '').replace(':', '')
    filename = os.path.join(folder_location, title + '.pdf')
    with open(filename, 'wb') as f:
        f.write(requests.get(paper['pdf']).content)


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:44<00:00, 11.23s/it]


## Getting other pages

In [289]:
from selenium import webdriver
driver = webdriver.Chrome()

In [77]:
page3 = '{"data":[{"items":[{"aids":["542a2809dabfae5848aa17b8","53f42df3dabfaedd74d3e053","59489cb29ed5db31dcdd0565","562d526545cedb3398dc474d","562e261a45cedb33990beecb","53f4d81cdabfaef64977b5bf","53f66220dabfae6a71b63777"],"authors":[{"id":"542a2809dabfae5848aa17b8","name":"Yunbo Wang"},{"id":"53f42df3dabfaedd74d3e053","name":"Bo Liu"},{"id":"59489cb29ed5db31dcdd0565","name":"Jiajun Wu"},{"id":"562d526545cedb3398dc474d","name":"Yuke Zhu"},{"id":"562e261a45cedb33990beecb","name":"Simon S Du"},{"id":"53f4d81cdabfaef64977b5bf","name":"Li Fei-Fei"},{"id":"53f66220dabfae6a71b63777","name":"Joshua B Tenenbaum"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/579","headline":"We provided an end-to-end neural network named Dual Sequential Monte Carlo to solve continuous Partially Observable Markov Decision Processes, which has three advantages","id":"5ee07fdc9e795e01364da3ca","is_like":"false","lang":"en","mrt_link":"5f4e050dd4631523323817f3","num_citation":0,"num_like":0,"num_viewed":150,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0579.pdf","picture":"https://cloud-api.scholarcy.com/images/0579.pdf_2k5gsklz_images_4g7wyxp6/img-001.png","title":"DualSMC: Tunneling Differentiable Filtering and Planning under Continuous POMDPs","year":2019},{"abstract":"  Conversational machine reading comprehension (MRC) has proven significantly more challenging compared to traditional MRC since it requires better utilization of conversation history. However, most existing approaches do not effectively capture conversation history and thus have trouble handling questions involving coreference or ellipsis. We propose a novel graph neural network (GNN) based model, namely GraphFlow, which captures conversational flow in the dialog. Specifically, we first propose a new approach to dynamically construct a question-aware context graph from passage text at each turn. We then present a novel flow mechanism to model the temporal dependencies in the sequence of context graphs. The proposed GraphFlow model shows superior performance compared to existing state-of-the-art methods. For instance, GraphFlow outperforms two recently proposed models on the CoQA benchmark dataset: FlowQA by 2.3% and SDNet by 0.7% on F1 score, respectively. In addition, visualization experiments show that our proposed model can better mimic the human reasoning process for conversational MRC compared to existing models. ","aids":["53f4339fdabfaedce5512fc6","562d97d045cedb3398e741ce","53f4ab9cdabfaecc2877b55a"],"authors":[{"id":"53f4339fdabfaedce5512fc6","name":"Chen Yu"},{"id":"562d97d045cedb3398e741ce","name":"Wu Lingfei"},{"id":"53f4ab9cdabfaecc2877b55a","name":"Zaki Mohammed J."}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/171","headline":"Interpretability analysis shows that the model can better mimic the human reasoning process for conversational machine comprehension compared to existing models","id":"5d4409be3a55acdd76472372","is_like":"false","mrt_link":"5f4e0503d4631523323816c3","num_citation":0,"num_like":0,"num_viewed":147,"pdf":"https://static.aminer.cn/upload/pdf/1882/1022/313/5d4409be3a55acdd76472372_0.pdf","picture":"https://cloud-api.scholarcy.com/images/5d4409be3a55acdd76472372_0.pdf_gqj9971u_images_5n_wq61y/img-001.png","title":"GraphFlow: Exploiting Conversation Flow with Graph Neural Networks for\n  Conversational Machine Comprehension","year":2019},{"aids":["542cbe96dabfae478c1973be","5609a30d45ce1e595f86e81d","562f1e3f45cedb339953a644","5440634edabfae805a6b798d","53f43b60dabfaec09f1adf5c"],"authors":[{"id":"542cbe96dabfae478c1973be","name":"Xuan Lin"},{"id":"5609a30d45ce1e595f86e81d","name":"Zhe Quan"},{"id":"562f1e3f45cedb339953a644","name":"Zhi-Jie Wang"},{"id":"5440634edabfae805a6b798d","name":"Tengfei Ma"},{"id":"53f43b60dabfaec09f1adf5c","name":"Xiangxiang Zeng"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/380","headline":"To address the above limitations, we propose an end-to-end framework, called Knowledge Graph Neural Network, to resolve the DDI prediction","id":"5ef96b048806af6ef2772133","is_like":"false","mrt_link":"5f4e0508d463152332381751","num_citation":0,"num_like":0,"num_viewed":147,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0380.pdf","picture":"https://cloud-api.scholarcy.com/images/0380.pdf_0rg7yu1__images_ob6beeah/img-001.png","resources":{},"title":"KGNN: Knowledge Graph Neural Network for Drug-Drug Interaction Prediction","year":2020},{"abstract":"  Accurate local-level poverty measurement is an essential task for governments and humanitarian organizations to track the progress towards improving livelihoods and distribute scarce resources. Recent computer vision advances in using satellite imagery to predict poverty have shown increasing accuracy, but they do not generate features that are interpretable to policymakers, inhibiting adoption by practitioners. Here we demonstrate an interpretable computational framework to accurately predict poverty at a local level by applying object detectors to high resolution (30cm) satellite images. Using the weighted counts of objects as features, we achieve 0.539 Pearson\'s r^2 in predicting village-level poverty in Uganda, a 31% improvement over existing (and less interpretable) benchmarks. Feature importance and ablation analysis reveal intuitive relationships between object counts and poverty predictions. Our results suggest that interpretability does not have to come at the cost of performance, at least in this important domain. ","aids":["5611f0ad45ce1e5962a380f8","53f433bfdabfaeee229903db","53f45f1edabfaee1c0b4d38c","5487fce8dabfae8a11fb3fcf","53f4c9ecdabfaee57f78001f"],"authors":[{"id":"5611f0ad45ce1e5962a380f8","name":"Ayush Kumar"},{"id":"53f433bfdabfaeee229903db","name":"Uzkent Burak"},{"id":"53f45f1edabfaee1c0b4d38c","name":"Burke Marshall"},{"id":"5487fce8dabfae8a11fb3fcf","name":"Lobell David"},{"id":"53f4c9ecdabfaee57f78001f","name":"Ermon Stefano"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/608","headline":"Our results offer a promising approach for generating interpretable poverty predictions for important livelihood outcomes, even in settings with limited training data","id":"5e3be3c33a55ac29c4ae7d26","is_like":"false","mrt_link":"5f4e0507d463152332381737","num_citation":8,"num_like":0,"num_viewed":146,"pdf":"https://static.aminer.cn/storage/pdf/arxiv/20/2002/2002.01612.pdf","picture":"https://cloud-api.scholarcy.com/images/2002.01612.pdf_psv7o7yk_images_n1par0u1/img-001.png","title":"Generating Interpretable Poverty Maps using Object Detection in\n  Satellite Images","year":2020},{"aids":["562fc2fe45cedb33997ccd12","53f431f4dabfaeb2ac023f3d","56066dff45cedb339687dcf2","53f42fdadabfaee4dc738419"],"authors":[{"id":"562fc2fe45cedb33997ccd12","name":"Yongrui Chen"},{"id":"53f431f4dabfaeb2ac023f3d","name":"Huiying Li"},{"id":"56066dff45cedb339687dcf2","name":"Yuncheng Hua"},{"id":"53f42fdadabfaee4dc738419","name":"Guilin Qi"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/519","headline":"The experimental results showed that our approach achieved superior results than the existing methods on complex questions, and produced competitive results on other simple question based datasets","id":"5ef96b048806af6ef27721d3","is_like":"false","mrt_link":"5f4e050dd4631523323817df","num_citation":1,"num_like":0,"num_viewed":141,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0519.pdf","picture":"https://cloud-api.scholarcy.com/images/0519.pdf_5fjcozbu_images_v9e06ofm/img-001.png","title":"Formal Query Building with Query Structure Prediction for Complex Question Answering over Knowledge Base","year":2020},{"abstract":"  In this paper, we study the two-facility location game on a line with optional preference where the acceptable set of facilities for each agent could be different and an agent\'s cost is his distance to the closest facility within his acceptable set. The objective is to minimize the total cost of all agents while achieving strategyproofness. We design a deterministic strategyproof mechanism for the problem with approximation ratio of 2.75, improving upon the earlier best ratio of n/2+1. ","aids":["5484aa67dabfaed7b5fa1aff","54879930dabfaed7b5fa31c0"],"authors":[{"id":"5484aa67dabfaed7b5fa1aff","name":"Li Minming"},{"id":"54879930dabfaed7b5fa31c0","name":"Lu Pinyan"},{"name":"Yao Yuhao"},{"name":"Zhang Jialin"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/34","headline":"It is an interesting open question to design a strategyproof mechanism with constant approximation ratio for the facility location game with three or more facilities","id":"5d36dcc13a55ac954df90e5a","is_like":"false","mrt_link":"5f4e0505d4631523323816f5","num_citation":0,"num_like":0,"num_viewed":141,"pdf":"https://static.aminer.cn/storage/pdf/arxiv/19/1907/1907.08918.pdf","picture":"https://cloud-api.scholarcy.com/images/1907.08918.pdf_t8hpy5_o_images_ng8r0fw0/img-001.png","title":"Strategyproof Mechanism for Two Heterogeneous Facilities with Constant\n  Approximation Ratio","year":2019},{"aids":["53f3b0abdabfae4b34b1a285","5432061edabfae8cc1c0a456","5603bce945cedb339622c268","543074cedabfaed7c7c59851","5606e4b545cedb3396987add","562da30845cedb3398e94041","53f47977dabfae8a6845b643"],"authors":[{"id":"53f3b0abdabfae4b34b1a285","name":"Dongxiao He"},{"id":"5432061edabfae8cc1c0a456","name":"Lu Zhai"},{"id":"5603bce945cedb339622c268","name":"Zhigang Li"},{"id":"543074cedabfaed7c7c59851","name":"Di Jin"},{"id":"5606e4b545cedb3396987add","name":"Liang Yang"},{"id":"562da30845cedb3398e94041","name":"Yuxiao Huang"},{"id":"53f47977dabfae8a6845b643","name":"Philip S. Yu"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/459","headline":"We propose to use the adversarial idea on the representation mechanism, i.e. on the encoding mechanism under the framework of autoencoder","id":"5ef96b048806af6ef2772198","is_like":"false","mrt_link":"5f4e0501d46315233238167c","num_citation":0,"num_like":0,"num_viewed":141,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0459.pdf","picture":"https://cloud-api.scholarcy.com/images/0459.pdf_f4dq6f80_images_rd9117i2/img-001.png","title":"Adversarial Mutual Information Learning for Network Embedding","year":2020},{"abstract":"  The optimization of expensive to evaluate, black-box, mixed-variable functions, i.e. functions that have continuous and discrete inputs, is a difficult and yet pervasive problem in science and engineering. In Bayesian optimization (BO), special cases of this problem that consider fully continuous or fully discrete domains have been widely studied. However, few methods exist for mixed-variable domains. In this paper, we introduce MiVaBo, a novel BO algorithm for the efficient optimization of mixed-variable functions that combines a linear surrogate model based on expressive feature representations with Thompson sampling. We propose two methods to optimize its acquisition function, a challenging problem for mixed-variable domains, and we show that MiVaBo can handle complex constraints over the discrete part of the domain that other methods cannot take into account. Moreover, we provide the first convergence analysis of a mixed-variable BO algorithm. Finally, we show that MiVaBo is significantly more sample efficient than state-of-the-art mixed-variable BO algorithms on hyperparameter tuning tasks. ","aids":["53f7b16cdabfae9467dac519","5624e3c045ce1e5964b0ea24","53f433d5dabfaedce551583c","53f48b19dabfaea6f277b55f"],"authors":[{"id":"53f7b16cdabfae9467dac519","name":"Daxberger Erik"},{"id":"5624e3c045ce1e5964b0ea24","name":"Makarova Anastasia"},{"id":"53f433d5dabfaedce551583c","name":"Turchetta Matteo"},{"id":"53f48b19dabfaea6f277b55f","name":"Krause Andreas"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/365","headline":"We propose MIVABO, a simple yet effective method for efficient optimization of expensive-toevaluate mixed-variable black-box objective functions, combining a linear model of expressive features with Thompson sampling","id":"5d1c7ccb3a55ac8c230aa9b9","is_like":"false","mrt_link":"5f4e050ad463152332381784","num_citation":4,"num_like":0,"num_viewed":140,"pdf":"https://static.aminer.cn/storage/pdf/arxiv/19/1907/1907.01329.pdf","picture":"https://cloud-api.scholarcy.com/images/1907.01329.pdf_pu7pz923_images_n3mkqffc/img-001.png","title":"Mixed-Variable Bayesian Optimization","year":2019},{"aids":["542a8ae3dabfae2b4e1153cf","53f3a52adabfae4b34ad40c8","562ab5af45cedb33989474ac","562d2e3a45cedb3398d7d59c","560d12c945cedb33975a2887","56078c2e45cedb3396af3338","562506cd45cedb3398570152"],"authors":[{"id":"542a8ae3dabfae2b4e1153cf","name":"Run Wang"},{"id":"53f3a52adabfae4b34ad40c8","name":"Felix Juefei-Xu"},{"id":"562ab5af45cedb33989474ac","name":"Lei Ma"},{"id":"562d2e3a45cedb3398d7d59c","name":"Xiaofei Xie"},{"id":"560d12c945cedb33975a2887","name":"Yihao Huang"},{"id":"56078c2e45cedb3396af3338","name":"Jian Wang"},{"id":"562506cd45cedb3398570152","name":"Yang Liu"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/476","headline":"We find that FakeSpotter achieves a better balance between precision and recall on four types of fake faces from Figure 3","id":"5ef96b048806af6ef27721af","is_like":"false","mrt_link":"5f4e0506d463152332381719","num_citation":3,"num_like":0,"num_viewed":135,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0476.pdf","picture":"https://cloud-api.scholarcy.com/images/0476.pdf_v0gitdok_images_zi7rv_d5/img-001.png","title":"FakeSpotter: A Simple yet Robust Baseline for Spotting AI-Synthesized Fake Faces","year":2020},{"abstract":"  Bipartite b-matching is fundamental in algorithm design, and has been widely applied into economic markets, labor markets, etc. These practical problems usually exhibit two distinct features: large-scale and dynamic, which requires the matching algorithm to be repeatedly executed at regular intervals. However, existing exact and approximate algorithms usually fail in such settings due to either requiring intolerable running time or too much computation resource. To address this issue, we propose \\texttt{NeuSearcher} which leverages the knowledge learned from previously instances to solve new problem instances. Specifically, we design a multichannel graph neural network to predict the threshold of the matched edges weights, by which the search region could be significantly reduced. We further propose a parallel heuristic search algorithm to iteratively improve the solution quality until convergence. Experiments on both open and industrial datasets demonstrate that \\texttt{NeuSearcher} can speed up 2 to 3 times while achieving exactly the same matching solution compared with the state-of-the-art approximation approaches. ","aids":["53f44ff9dabfaee4dc7f7cfd","53f43454dabfaedce551b305","560fe71345ce1e5961f0fe0f","53f43d81dabfaee1c0ad528b","542bc160dabfae1bbfd1780e","5d43d51f7390bff0db5ff255","53f43449dabfaeecd694b202"],"authors":[{"id":"53f44ff9dabfaee4dc7f7cfd","name":"Hao Xiaotian"},{"name":"Jin Junqi"},{"id":"53f43454dabfaedce551b305","name":"Hao Jianye"},{"name":"Li Jin"},{"id":"560fe71345ce1e5961f0fe0f","name":"Wang Weixun"},{"name":"Ma Yi"},{"id":"53f43d81dabfaee1c0ad528b","name":"Zheng Zhenzhe"},{"id":"542bc160dabfae1bbfd1780e","name":"Li Han"},{"id":"5d43d51f7390bff0db5ff255","name":"Xu Jian"},{"id":"53f43449dabfaeecd694b202","name":"Gai Kun"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/475","headline":"Our NeuSearcher transfers knowledge learned from previous solved instances to save more than 50% of the computing time","id":"5eba73be91e01108d77cf63e","is_like":"false","mrt_link":"5f4e0501d463152332381680","num_citation":0,"num_like":1,"num_viewed":132,"pdf":"https://static.aminer.cn/storage/pdf/arxiv/20/2005/2005.04355.pdf","picture":"https://cloud-api.scholarcy.com/images/2005.04355.pdf_jjsj22rk_images_fokpiz5n/img-001.png","title":"Learning to Accelerate Heuristic Searching for Large-Scale Maximum\n  Weighted b-Matching Problems in Online Advertising","year":2020},{"aids":["5621f6d445cedb339837892a","5408f140dabfae8faa666053","5d43d53b7390bff0db600282"],"authors":[{"id":"5621f6d445cedb339837892a","name":"Ziwei Wang"},{"id":"5408f140dabfae8faa666053","name":"Zi Huang"},{"id":"5d43d53b7390bff0db600282","name":"Yadan Luo"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/92","headline":"We propose a human consensus-oriented model for image captioning","id":"5ef96b048806af6ef2772038","is_like":"false","mrt_link":"5f4e0511d463152332381840","num_citation":2,"num_like":0,"num_viewed":131,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0092.pdf","picture":"https://cloud-api.scholarcy.com/images/0092.pdf_5f4l0eh3_images_12wgs2uw/img-001.png","title":"Human Consensus-Oriented Image Captioning","year":2020},{"abstract":"  Inferring topological characteristics of complex networks from observed data is critical to understand the dynamical behavior of networked systems, ranging from the Internet and the World Wide Web to biological networks and social networks. Prior studies usually focus on the structure-based estimation to infer network sizes, degree distributions, average degrees, and more. Little effort attempted to estimate the specific degree of each vertex from a sampled induced graph, which prevents us from measuring the lethality of nodes in protein networks and influencers in social networks. The current approaches dramatically fail for a tiny sampled induced graph and require a specific sampling method and a large sample size. These approaches neglect information of the vertex state, representing the dynamical behavior of the networked system, such as the biomass of species or expression of a gene, which is useful for degree estimation. We fill this gap by developing a framework to infer individual vertex degrees using both information of the sampled topology and vertex state. We combine the mean-field theory with combinatorial optimization to learn vertex degrees. Experimental results on real networks with a variety of dynamics demonstrate that our framework can produce reliable degree estimates and dramatically improve existing link prediction methods by replacing the sampled degrees with our estimated degrees. ","aids":["53f46917dabfaeecd6a17760","53f47d63dabfaee4dc8a59e2","53f39be6dabfae4b34a99107"],"authors":[{"id":"53f46917dabfaeecd6a17760","name":"Jiang Chunheng"},{"id":"53f47d63dabfaee4dc8a59e2","name":"Gao Jianxi"},{"id":"53f39be6dabfae4b34a99107","name":"Magdon-Ismail Malik"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/457","headline":"We show that our approaches can tolerate high level of noise added to the observed steadystates, as well as model misspecification in the form of noise added to the parameters in the dynamical system","id":"5ea16b3491e011fa08b8fb55","is_like":"false","mrt_link":"5f4e0505d4631523323816f7","num_citation":0,"num_like":0,"num_viewed":130,"pdf":"https://static.aminer.cn/storage/pdf/arxiv/20/2004/2004.10546.pdf","picture":"https://cloud-api.scholarcy.com/images/2004.10546.pdf_pus3hpx__images_r9rfd34h/img-001.png","title":"Inferring Degrees from Incomplete Networks and Nonlinear Dynamics","year":2020},{"aids":["5d50c3797390bff0db2a5114","542be28cdabfae216e61cab1","53f38139dabfae4b349f8a2b","53f58dc0dabfaed67df8067c","53f46741dabfaee4dc851a50","53f43267dabfaeb1a7bcb584","562d27e145cedb3398d6c781"],"authors":[{"id":"5d50c3797390bff0db2a5114","name":"Dor Atzmon"},{"id":"542be28cdabfae216e61cab1","name":"Jiaoyang Li"},{"id":"53f38139dabfae4b349f8a2b","name":"Ariel Felner"},{"id":"53f58dc0dabfaed67df8067c","name":"Eliran Nachmani"},{"id":"53f46741dabfaee4dc851a50","name":"Shahaf S. Shperberg"},{"id":"53f43267dabfaeb1a7bcb584","name":"Nathan R. Sturtevant"},{"id":"562d27e145cedb3398d6c781","name":"Sven Koenig"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/562","headline":"We introduced the multi-directional search algorithm Meet in the Middle* that optimally solves Multi-Agent Meeting problem","id":"5ef96b048806af6ef2772208","is_like":"false","mrt_link":"5f4e0509d463152332381779","num_citation":1,"num_like":0,"num_viewed":130,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0562.pdf","picture":"https://cloud-api.scholarcy.com/images/0562.pdf_olm0ktwd_images_edq1rnau/img-001.png","title":"Multi-Directional Heuristic Search","year":2020},{"aids":["542e1ce9dabfae11fc4c5dd0","53f4399fdabfaee2a1d04811","561d1c8b45cedb339803f3f2","53f42b4adabfaeb22f3e6b4a","560cd53c45cedb339757a831","540543afdabfae91d3fcd837","540683ccdabfae44f083b82c"],"authors":[{"id":"542e1ce9dabfae11fc4c5dd0","name":"Changbin Shao"},{"id":"53f4399fdabfaee2a1d04811","name":"Jing Huo"},{"id":"561d1c8b45cedb339803f3f2","name":"Lei Qi"},{"id":"53f42b4adabfaeb22f3e6b4a","name":"Zhen-Hua Feng"},{"id":"560cd53c45cedb339757a831","name":"Wenbin Li"},{"id":"540543afdabfae91d3fcd837","name":"Chuanqi Dong"},{"id":"540683ccdabfae44f083b82c","name":"Yang Gao"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/93","headline":"The experimental results demonstrated the merits of our Biased Feature Learning method as well as its generalization capability with different network architectures and loss functions","id":"5ef96b048806af6ef2772017","is_like":"false","mrt_link":"5f4e0505d463152332381702","num_citation":0,"num_like":0,"num_viewed":127,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0093.pdf","picture":"https://cloud-api.scholarcy.com/images/0093.pdf_xb8wik8h_images_hyj4wfzq/img-001.png","title":"Biased Feature Learning for Occlusion Invariant Face Recognition","year":2020},{"abstract":"  Unsupervised image captioning with no annotations is an emerging challenge in computer vision, where the existing arts usually adopt GAN (Generative Adversarial Networks) models. In this paper, we propose a novel memory-based network rather than GAN, named Recurrent Relational Memory Network ($R^2M$). Unlike complicated and sensitive adversarial learning that non-ideally performs for long sentence generation, $R^2M$ implements a concepts-to-sentence memory translator through two-stage memory mechanisms: fusion and recurrent memories, correlating the relational reasoning between common visual concepts and the generated words for long periods. $R^2M$ encodes visual context through unsupervised training on images, while enabling the memory to learn from irrelevant textual corpus via supervised fashion. Our solution enjoys less learnable parameters and higher computational efficiency than GAN-based methods, which heavily bear parameter sensitivity. We experimentally validate the superiority of $R^2M$ than state-of-the-arts on all benchmark datasets. ","aids":["542c00fbdabfae2b4e1cbbb2","560dadd545cedb3397609cf0","56cb1891c35f4f3c6565176f"],"authors":[{"id":"542c00fbdabfae2b4e1cbbb2","name":"Dan Guo"},{"id":"560dadd545cedb3397609cf0","name":"Yang Wang"},{"name":"Peipei Song"},{"id":"56cb1891c35f4f3c6565176f","name":"Meng Wang"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/128","headline":"This paper proposes a novel recurrent relational memory network for unsupervised image captioning with low cost of supervision","id":"5ef476b691e01165a63bbbdc","is_like":"false","mrt_link":"5f4e0501d463152332381695","num_citation":0,"num_like":1,"num_viewed":126,"pdf":"https://static.aminer.cn/upload/pdf/884/764/28/5ef476b691e01165a63bbbdc_0.pdf","picture":"https://cloud-api.scholarcy.com/images/5ef476b691e01165a63bbbdc_0.pdf_4du9k826_images_u4qf5m3m/img-001.png","title":"Recurrent Relational Memory Network for Unsupervised Image Captioning","year":2020},{"abstract":"  A major challenge in inductive logic programming (ILP) is learning large programs. We argue that a key limitation of existing systems is that they use entailment to guide the hypothesis search. This approach is limited because entailment is a binary decision: a hypothesis either entails an example or does not, and there is no intermediate position. To address this limitation, we go beyond entailment and use \\emph{example-dependent} loss functions to guide the search, where a hypothesis can partially cover an example. We implement our idea in Brute, a new ILP system which uses best-first search, guided by an example-dependent loss function, to incrementally build programs. Our experiments on three diverse program synthesis domains (robot planning, string transformations, and ASCII art), show that Brute can substantially outperform existing ILP systems, both in terms of predictive accuracies and learning times, and can learn programs 20 times larger than state-of-the-art systems. ","aids":["5d50c38b7390bff0db2a607a","5d50c38f7390bff0db2a63f9"],"authors":[{"id":"5d50c38b7390bff0db2a607a","name":"Cropper Andrew"},{"id":"5d50c38f7390bff0db2a63f9","name":"Dumančić Sebastijan"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/287","headline":"We implemented our idea in Brute, a new inductive logic programming which first invents a library of predicates, including recursive predicates, and performs a best-first search informed by a given loss function to build a hypothesis using the library predicates","id":"5ea16b2b91e011fa08b8f8a2","is_like":"false","mrt_link":"5f4e050ed463152332381804","num_citation":1,"num_like":0,"num_viewed":126,"pdf":"https://static.aminer.cn/storage/pdf/arxiv/20/2004/2004.09855.pdf","picture":"https://cloud-api.scholarcy.com/images/2004.09855.pdf_tz3eo8xy_images_my_y_4ri/img-001.png","title":"Learning large logic programs by going beyond entailment","year":2020},{"aids":["542ead9adabfae4a9e494982","53f447cbdabfaedf435d1f03","53f466c9dabfaeb22f53efd7","53f45084dabfaeecd69d4b63","53f4926bdabfaedd74eb491b"],"authors":[{"id":"542ead9adabfae4a9e494982","name":"Saba Ahmadi"},{"id":"53f447cbdabfaedf435d1f03","name":"Faez Ahmed"},{"id":"53f466c9dabfaeb22f53efd7","name":"John P. Dickerson"},{"id":"53f45084dabfaeecd69d4b63","name":"Mark Fuge"},{"id":"53f4926bdabfaedd74eb491b","name":"Samir Khuller"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/1","headline":"We propose an algorithm that guarantees an optimal solution and converges faster than a proposed approach using a black-box industrial Mixed-Integer Quadratic Program solver","id":"5ef96b048806af6ef2771fd5","is_like":"false","mrt_link":"5f4e0509d46315233238176e","num_citation":1,"num_like":0,"num_viewed":126,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0001.pdf","picture":"https://cloud-api.scholarcy.com/images/0001.pdf_kin01g3p_images_ydd11zeb/img-001.png","title":"An Algorithm for Multi-Attribute Diverse Matching","year":2020},{"aids":["56311fc245cedb3399cb3327","53f3588fdabfae4b34965da0"],"authors":[{"id":"56311fc245cedb3399cb3327","name":"Siyu Lin"},{"id":"53f3588fdabfae4b34965da0","name":"Peter A. Beling"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/627","headline":"We propose an end-to-end optimal trade execution framework based on PPO","id":"5ef96b048806af6ef2772245","is_like":"false","mrt_link":"5f4e0517d4631523323818e3","num_citation":0,"num_like":0,"num_viewed":123,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0627.pdf","picture":"https://cloud-api.scholarcy.com/images/0627.pdf_72fuenkj_images_uqbq8fui/img-001.png","title":"An End-to-End Optimal Trade Execution Framework based on Proximal Policy Optimization","year":2020},{"abstract":"  Existing convergence analyses of Q-learning mostly focus on the vanilla stochastic gradient descent (SGD) type of updates. Despite the Adaptive Moment Estimation (Adam) has been commonly used for practical Q-learning algorithms, there has not been any convergence guarantee provided for Q-learning with such type of updates. In this paper, we first characterize the convergence rate for Q-AMSGrad, which is the Q-learning algorithm with AMSGrad update (a commonly adopted alternative of Adam for theoretical analysis). To further improve the performance, we propose to incorporate the momentum restart scheme to Q-AMSGrad, resulting in the so-called Q-AMSGradR algorithm. The convergence rate of Q-AMSGradR is also established. Our experiments on a linear quadratic regulator problem show that the two proposed Q-learning algorithms outperform the vanilla Q-learning with SGD updates. The two algorithms also exhibit significantly better performance than the DQN learning method over a batch of Atari 2600 games. ","aids":["5617a08e45ce1e5963e405e7","5618326645ce1e5963f93d64","5405a7ecdabfae450f3c22a6","53f4b8f6dabfaed83b77b422"],"authors":[{"id":"5617a08e45ce1e5963e405e7","name":"Bowen Weng"},{"id":"5618326645ce1e5963f93d64","name":"Huaqing Xiong"},{"id":"5405a7ecdabfae450f3c22a6","name":"Yingbin Liang"},{"id":"53f4b8f6dabfaed83b77b422","name":"Wei Zhang"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/422","headline":"We study two Q-learning algorithms with Adaptive Moment Estimation-type updates, and demonstrate their superior performance over the vanilla Q-learning and deep Q-Network algorithms through a linear quadratic regulator problem and a batch of 23 Atari 2600 games","id":"5ef96b048806af6ef27720e4","is_like":"false","mrt_link":"5f4e050ed463152332381805","num_citation":1,"num_like":0,"num_viewed":123,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0422.pdf","picture":"https://cloud-api.scholarcy.com/images/0422.pdf_ikn8agkw_images_86w4v40v/img-001.png","title":"Analysis of Q-learning with Adaptation and Momentum Restart for Gradient Descent","year":2020},{"aids":["53f35095dabfae4b34945bf1","5611f65745ce1e5962a51665","54875be8dabfae9b40134539"],"authors":[{"id":"53f35095dabfae4b34945bf1","name":"Andreas Niskanen"},{"id":"5611f65745ce1e5962a51665","name":"Daniel Neugebauer"},{"id":"54875be8dabfae9b40134539","name":"Matti JÃ¤rvisalo"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/257","headline":"We provided complexity results for credulous and skeptical controllability in control argumentation frameworks under five central semantics","id":"5ef96b048806af6ef27720b8","is_like":"false","mrt_link":"5f4e050ed463152332381810","num_citation":1,"num_like":0,"num_viewed":121,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0257.pdf","picture":"https://cloud-api.scholarcy.com/images/0257.pdf_5chbcjeh_images_uc88qivs/img-001.png","title":"Controllability of Control Argumentation Frameworks","year":2020}],"meta":{"context":"3596084E","time":"811.139271ms"},"succeed":"true","total":645}]}'
page3 = page3.replace('\\n', '').replace(':false', ':"false"').replace(':true', ':"true"')

In [78]:
json.loads(page3)

JSONDecodeError: Invalid control character at: line 1 column 3357 (char 3356)

In [79]:
page3 = {"data":[{"items":[{"aids":["542a2809dabfae5848aa17b8","53f42df3dabfaedd74d3e053","59489cb29ed5db31dcdd0565","562d526545cedb3398dc474d","562e261a45cedb33990beecb","53f4d81cdabfaef64977b5bf","53f66220dabfae6a71b63777"],"authors":[{"id":"542a2809dabfae5848aa17b8","name":"Yunbo Wang"},{"id":"53f42df3dabfaedd74d3e053","name":"Bo Liu"},{"id":"59489cb29ed5db31dcdd0565","name":"Jiajun Wu"},{"id":"562d526545cedb3398dc474d","name":"Yuke Zhu"},{"id":"562e261a45cedb33990beecb","name":"Simon S Du"},{"id":"53f4d81cdabfaef64977b5bf","name":"Li Fei-Fei"},{"id":"53f66220dabfae6a71b63777","name":"Joshua B Tenenbaum"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/579","headline":"We provided an end-to-end neural network named Dual Sequential Monte Carlo to solve continuous Partially Observable Markov Decision Processes, which has three advantages","id":"5ee07fdc9e795e01364da3ca","is_like":"false","lang":"en","mrt_link":"5f4e050dd4631523323817f3","num_citation":0,"num_like":0,"num_viewed":150,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0579.pdf","picture":"https://cloud-api.scholarcy.com/images/0579.pdf_2k5gsklz_images_4g7wyxp6/img-001.png","title":"DualSMC: Tunneling Differentiable Filtering and Planning under Continuous POMDPs","year":2019},{"abstract":"  Conversational machine reading comprehension (MRC) has proven significantly more challenging compared to traditional MRC since it requires better utilization of conversation history. However, most existing approaches do not effectively capture conversation history and thus have trouble handling questions involving coreference or ellipsis. We propose a novel graph neural network (GNN) based model, namely GraphFlow, which captures conversational flow in the dialog. Specifically, we first propose a new approach to dynamically construct a question-aware context graph from passage text at each turn. We then present a novel flow mechanism to model the temporal dependencies in the sequence of context graphs. The proposed GraphFlow model shows superior performance compared to existing state-of-the-art methods. For instance, GraphFlow outperforms two recently proposed models on the CoQA benchmark dataset: FlowQA by 2.3% and SDNet by 0.7% on F1 score, respectively. In addition, visualization experiments show that our proposed model can better mimic the human reasoning process for conversational MRC compared to existing models. ","aids":["53f4339fdabfaedce5512fc6","562d97d045cedb3398e741ce","53f4ab9cdabfaecc2877b55a"],"authors":[{"id":"53f4339fdabfaedce5512fc6","name":"Chen Yu"},{"id":"562d97d045cedb3398e741ce","name":"Wu Lingfei"},{"id":"53f4ab9cdabfaecc2877b55a","name":"Zaki Mohammed J."}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/171","headline":"Interpretability analysis shows that the model can better mimic the human reasoning process for conversational machine comprehension compared to existing models","id":"5d4409be3a55acdd76472372","is_like":"false","mrt_link":"5f4e0503d4631523323816c3","num_citation":0,"num_like":0,"num_viewed":147,"pdf":"https://static.aminer.cn/upload/pdf/1882/1022/313/5d4409be3a55acdd76472372_0.pdf","picture":"https://cloud-api.scholarcy.com/images/5d4409be3a55acdd76472372_0.pdf_gqj9971u_images_5n_wq61y/img-001.png","title":"GraphFlow: Exploiting Conversation Flow with Graph Neural Networks for\n  Conversational Machine Comprehension","year":2019},{"aids":["542cbe96dabfae478c1973be","5609a30d45ce1e595f86e81d","562f1e3f45cedb339953a644","5440634edabfae805a6b798d","53f43b60dabfaec09f1adf5c"],"authors":[{"id":"542cbe96dabfae478c1973be","name":"Xuan Lin"},{"id":"5609a30d45ce1e595f86e81d","name":"Zhe Quan"},{"id":"562f1e3f45cedb339953a644","name":"Zhi-Jie Wang"},{"id":"5440634edabfae805a6b798d","name":"Tengfei Ma"},{"id":"53f43b60dabfaec09f1adf5c","name":"Xiangxiang Zeng"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/380","headline":"To address the above limitations, we propose an end-to-end framework, called Knowledge Graph Neural Network, to resolve the DDI prediction","id":"5ef96b048806af6ef2772133","is_like":"false","mrt_link":"5f4e0508d463152332381751","num_citation":0,"num_like":0,"num_viewed":147,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0380.pdf","picture":"https://cloud-api.scholarcy.com/images/0380.pdf_0rg7yu1__images_ob6beeah/img-001.png","resources":{},"title":"KGNN: Knowledge Graph Neural Network for Drug-Drug Interaction Prediction","year":2020},{"abstract":"  Accurate local-level poverty measurement is an essential task for governments and humanitarian organizations to track the progress towards improving livelihoods and distribute scarce resources. Recent computer vision advances in using satellite imagery to predict poverty have shown increasing accuracy, but they do not generate features that are interpretable to policymakers, inhibiting adoption by practitioners. Here we demonstrate an interpretable computational framework to accurately predict poverty at a local level by applying object detectors to high resolution (30cm) satellite images. Using the weighted counts of objects as features, we achieve 0.539 Pearson\'s r^2 in predicting village-level poverty in Uganda, a 31% improvement over existing (and less interpretable) benchmarks. Feature importance and ablation analysis reveal intuitive relationships between object counts and poverty predictions. Our results suggest that interpretability does not have to come at the cost of performance, at least in this important domain. ","aids":["5611f0ad45ce1e5962a380f8","53f433bfdabfaeee229903db","53f45f1edabfaee1c0b4d38c","5487fce8dabfae8a11fb3fcf","53f4c9ecdabfaee57f78001f"],"authors":[{"id":"5611f0ad45ce1e5962a380f8","name":"Ayush Kumar"},{"id":"53f433bfdabfaeee229903db","name":"Uzkent Burak"},{"id":"53f45f1edabfaee1c0b4d38c","name":"Burke Marshall"},{"id":"5487fce8dabfae8a11fb3fcf","name":"Lobell David"},{"id":"53f4c9ecdabfaee57f78001f","name":"Ermon Stefano"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/608","headline":"Our results offer a promising approach for generating interpretable poverty predictions for important livelihood outcomes, even in settings with limited training data","id":"5e3be3c33a55ac29c4ae7d26","is_like":"false","mrt_link":"5f4e0507d463152332381737","num_citation":8,"num_like":0,"num_viewed":146,"pdf":"https://static.aminer.cn/storage/pdf/arxiv/20/2002/2002.01612.pdf","picture":"https://cloud-api.scholarcy.com/images/2002.01612.pdf_psv7o7yk_images_n1par0u1/img-001.png","title":"Generating Interpretable Poverty Maps using Object Detection in\n  Satellite Images","year":2020},{"aids":["562fc2fe45cedb33997ccd12","53f431f4dabfaeb2ac023f3d","56066dff45cedb339687dcf2","53f42fdadabfaee4dc738419"],"authors":[{"id":"562fc2fe45cedb33997ccd12","name":"Yongrui Chen"},{"id":"53f431f4dabfaeb2ac023f3d","name":"Huiying Li"},{"id":"56066dff45cedb339687dcf2","name":"Yuncheng Hua"},{"id":"53f42fdadabfaee4dc738419","name":"Guilin Qi"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/519","headline":"The experimental results showed that our approach achieved superior results than the existing methods on complex questions, and produced competitive results on other simple question based datasets","id":"5ef96b048806af6ef27721d3","is_like":"false","mrt_link":"5f4e050dd4631523323817df","num_citation":1,"num_like":0,"num_viewed":141,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0519.pdf","picture":"https://cloud-api.scholarcy.com/images/0519.pdf_5fjcozbu_images_v9e06ofm/img-001.png","title":"Formal Query Building with Query Structure Prediction for Complex Question Answering over Knowledge Base","year":2020},{"abstract":"  In this paper, we study the two-facility location game on a line with optional preference where the acceptable set of facilities for each agent could be different and an agent\'s cost is his distance to the closest facility within his acceptable set. The objective is to minimize the total cost of all agents while achieving strategyproofness. We design a deterministic strategyproof mechanism for the problem with approximation ratio of 2.75, improving upon the earlier best ratio of n/2+1. ","aids":["5484aa67dabfaed7b5fa1aff","54879930dabfaed7b5fa31c0"],"authors":[{"id":"5484aa67dabfaed7b5fa1aff","name":"Li Minming"},{"id":"54879930dabfaed7b5fa31c0","name":"Lu Pinyan"},{"name":"Yao Yuhao"},{"name":"Zhang Jialin"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/34","headline":"It is an interesting open question to design a strategyproof mechanism with constant approximation ratio for the facility location game with three or more facilities","id":"5d36dcc13a55ac954df90e5a","is_like":"false","mrt_link":"5f4e0505d4631523323816f5","num_citation":0,"num_like":0,"num_viewed":141,"pdf":"https://static.aminer.cn/storage/pdf/arxiv/19/1907/1907.08918.pdf","picture":"https://cloud-api.scholarcy.com/images/1907.08918.pdf_t8hpy5_o_images_ng8r0fw0/img-001.png","title":"Strategyproof Mechanism for Two Heterogeneous Facilities with Constant\n  Approximation Ratio","year":2019},{"aids":["53f3b0abdabfae4b34b1a285","5432061edabfae8cc1c0a456","5603bce945cedb339622c268","543074cedabfaed7c7c59851","5606e4b545cedb3396987add","562da30845cedb3398e94041","53f47977dabfae8a6845b643"],"authors":[{"id":"53f3b0abdabfae4b34b1a285","name":"Dongxiao He"},{"id":"5432061edabfae8cc1c0a456","name":"Lu Zhai"},{"id":"5603bce945cedb339622c268","name":"Zhigang Li"},{"id":"543074cedabfaed7c7c59851","name":"Di Jin"},{"id":"5606e4b545cedb3396987add","name":"Liang Yang"},{"id":"562da30845cedb3398e94041","name":"Yuxiao Huang"},{"id":"53f47977dabfae8a6845b643","name":"Philip S. Yu"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/459","headline":"We propose to use the adversarial idea on the representation mechanism, i.e. on the encoding mechanism under the framework of autoencoder","id":"5ef96b048806af6ef2772198","is_like":"false","mrt_link":"5f4e0501d46315233238167c","num_citation":0,"num_like":0,"num_viewed":141,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0459.pdf","picture":"https://cloud-api.scholarcy.com/images/0459.pdf_f4dq6f80_images_rd9117i2/img-001.png","title":"Adversarial Mutual Information Learning for Network Embedding","year":2020},{"abstract":"  The optimization of expensive to evaluate, black-box, mixed-variable functions, i.e. functions that have continuous and discrete inputs, is a difficult and yet pervasive problem in science and engineering. In Bayesian optimization (BO), special cases of this problem that consider fully continuous or fully discrete domains have been widely studied. However, few methods exist for mixed-variable domains. In this paper, we introduce MiVaBo, a novel BO algorithm for the efficient optimization of mixed-variable functions that combines a linear surrogate model based on expressive feature representations with Thompson sampling. We propose two methods to optimize its acquisition function, a challenging problem for mixed-variable domains, and we show that MiVaBo can handle complex constraints over the discrete part of the domain that other methods cannot take into account. Moreover, we provide the first convergence analysis of a mixed-variable BO algorithm. Finally, we show that MiVaBo is significantly more sample efficient than state-of-the-art mixed-variable BO algorithms on hyperparameter tuning tasks. ","aids":["53f7b16cdabfae9467dac519","5624e3c045ce1e5964b0ea24","53f433d5dabfaedce551583c","53f48b19dabfaea6f277b55f"],"authors":[{"id":"53f7b16cdabfae9467dac519","name":"Daxberger Erik"},{"id":"5624e3c045ce1e5964b0ea24","name":"Makarova Anastasia"},{"id":"53f433d5dabfaedce551583c","name":"Turchetta Matteo"},{"id":"53f48b19dabfaea6f277b55f","name":"Krause Andreas"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/365","headline":"We propose MIVABO, a simple yet effective method for efficient optimization of expensive-toevaluate mixed-variable black-box objective functions, combining a linear model of expressive features with Thompson sampling","id":"5d1c7ccb3a55ac8c230aa9b9","is_like":"false","mrt_link":"5f4e050ad463152332381784","num_citation":4,"num_like":0,"num_viewed":140,"pdf":"https://static.aminer.cn/storage/pdf/arxiv/19/1907/1907.01329.pdf","picture":"https://cloud-api.scholarcy.com/images/1907.01329.pdf_pu7pz923_images_n3mkqffc/img-001.png","title":"Mixed-Variable Bayesian Optimization","year":2019},{"aids":["542a8ae3dabfae2b4e1153cf","53f3a52adabfae4b34ad40c8","562ab5af45cedb33989474ac","562d2e3a45cedb3398d7d59c","560d12c945cedb33975a2887","56078c2e45cedb3396af3338","562506cd45cedb3398570152"],"authors":[{"id":"542a8ae3dabfae2b4e1153cf","name":"Run Wang"},{"id":"53f3a52adabfae4b34ad40c8","name":"Felix Juefei-Xu"},{"id":"562ab5af45cedb33989474ac","name":"Lei Ma"},{"id":"562d2e3a45cedb3398d7d59c","name":"Xiaofei Xie"},{"id":"560d12c945cedb33975a2887","name":"Yihao Huang"},{"id":"56078c2e45cedb3396af3338","name":"Jian Wang"},{"id":"562506cd45cedb3398570152","name":"Yang Liu"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/476","headline":"We find that FakeSpotter achieves a better balance between precision and recall on four types of fake faces from Figure 3","id":"5ef96b048806af6ef27721af","is_like":"false","mrt_link":"5f4e0506d463152332381719","num_citation":3,"num_like":0,"num_viewed":135,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0476.pdf","picture":"https://cloud-api.scholarcy.com/images/0476.pdf_v0gitdok_images_zi7rv_d5/img-001.png","title":"FakeSpotter: A Simple yet Robust Baseline for Spotting AI-Synthesized Fake Faces","year":2020},{"abstract":"  Bipartite b-matching is fundamental in algorithm design, and has been widely applied into economic markets, labor markets, etc. These practical problems usually exhibit two distinct features: large-scale and dynamic, which requires the matching algorithm to be repeatedly executed at regular intervals. However, existing exact and approximate algorithms usually fail in such settings due to either requiring intolerable running time or too much computation resource. To address this issue, we propose \\texttt{NeuSearcher} which leverages the knowledge learned from previously instances to solve new problem instances. Specifically, we design a multichannel graph neural network to predict the threshold of the matched edges weights, by which the search region could be significantly reduced. We further propose a parallel heuristic search algorithm to iteratively improve the solution quality until convergence. Experiments on both open and industrial datasets demonstrate that \\texttt{NeuSearcher} can speed up 2 to 3 times while achieving exactly the same matching solution compared with the state-of-the-art approximation approaches. ","aids":["53f44ff9dabfaee4dc7f7cfd","53f43454dabfaedce551b305","560fe71345ce1e5961f0fe0f","53f43d81dabfaee1c0ad528b","542bc160dabfae1bbfd1780e","5d43d51f7390bff0db5ff255","53f43449dabfaeecd694b202"],"authors":[{"id":"53f44ff9dabfaee4dc7f7cfd","name":"Hao Xiaotian"},{"name":"Jin Junqi"},{"id":"53f43454dabfaedce551b305","name":"Hao Jianye"},{"name":"Li Jin"},{"id":"560fe71345ce1e5961f0fe0f","name":"Wang Weixun"},{"name":"Ma Yi"},{"id":"53f43d81dabfaee1c0ad528b","name":"Zheng Zhenzhe"},{"id":"542bc160dabfae1bbfd1780e","name":"Li Han"},{"id":"5d43d51f7390bff0db5ff255","name":"Xu Jian"},{"id":"53f43449dabfaeecd694b202","name":"Gai Kun"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/475","headline":"Our NeuSearcher transfers knowledge learned from previous solved instances to save more than 50% of the computing time","id":"5eba73be91e01108d77cf63e","is_like":"false","mrt_link":"5f4e0501d463152332381680","num_citation":0,"num_like":1,"num_viewed":132,"pdf":"https://static.aminer.cn/storage/pdf/arxiv/20/2005/2005.04355.pdf","picture":"https://cloud-api.scholarcy.com/images/2005.04355.pdf_jjsj22rk_images_fokpiz5n/img-001.png","title":"Learning to Accelerate Heuristic Searching for Large-Scale Maximum\n  Weighted b-Matching Problems in Online Advertising","year":2020},{"aids":["5621f6d445cedb339837892a","5408f140dabfae8faa666053","5d43d53b7390bff0db600282"],"authors":[{"id":"5621f6d445cedb339837892a","name":"Ziwei Wang"},{"id":"5408f140dabfae8faa666053","name":"Zi Huang"},{"id":"5d43d53b7390bff0db600282","name":"Yadan Luo"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/92","headline":"We propose a human consensus-oriented model for image captioning","id":"5ef96b048806af6ef2772038","is_like":"false","mrt_link":"5f4e0511d463152332381840","num_citation":2,"num_like":0,"num_viewed":131,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0092.pdf","picture":"https://cloud-api.scholarcy.com/images/0092.pdf_5f4l0eh3_images_12wgs2uw/img-001.png","title":"Human Consensus-Oriented Image Captioning","year":2020},{"abstract":"  Inferring topological characteristics of complex networks from observed data is critical to understand the dynamical behavior of networked systems, ranging from the Internet and the World Wide Web to biological networks and social networks. Prior studies usually focus on the structure-based estimation to infer network sizes, degree distributions, average degrees, and more. Little effort attempted to estimate the specific degree of each vertex from a sampled induced graph, which prevents us from measuring the lethality of nodes in protein networks and influencers in social networks. The current approaches dramatically fail for a tiny sampled induced graph and require a specific sampling method and a large sample size. These approaches neglect information of the vertex state, representing the dynamical behavior of the networked system, such as the biomass of species or expression of a gene, which is useful for degree estimation. We fill this gap by developing a framework to infer individual vertex degrees using both information of the sampled topology and vertex state. We combine the mean-field theory with combinatorial optimization to learn vertex degrees. Experimental results on real networks with a variety of dynamics demonstrate that our framework can produce reliable degree estimates and dramatically improve existing link prediction methods by replacing the sampled degrees with our estimated degrees. ","aids":["53f46917dabfaeecd6a17760","53f47d63dabfaee4dc8a59e2","53f39be6dabfae4b34a99107"],"authors":[{"id":"53f46917dabfaeecd6a17760","name":"Jiang Chunheng"},{"id":"53f47d63dabfaee4dc8a59e2","name":"Gao Jianxi"},{"id":"53f39be6dabfae4b34a99107","name":"Magdon-Ismail Malik"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/457","headline":"We show that our approaches can tolerate high level of noise added to the observed steadystates, as well as model misspecification in the form of noise added to the parameters in the dynamical system","id":"5ea16b3491e011fa08b8fb55","is_like":"false","mrt_link":"5f4e0505d4631523323816f7","num_citation":0,"num_like":0,"num_viewed":130,"pdf":"https://static.aminer.cn/storage/pdf/arxiv/20/2004/2004.10546.pdf","picture":"https://cloud-api.scholarcy.com/images/2004.10546.pdf_pus3hpx__images_r9rfd34h/img-001.png","title":"Inferring Degrees from Incomplete Networks and Nonlinear Dynamics","year":2020},{"aids":["5d50c3797390bff0db2a5114","542be28cdabfae216e61cab1","53f38139dabfae4b349f8a2b","53f58dc0dabfaed67df8067c","53f46741dabfaee4dc851a50","53f43267dabfaeb1a7bcb584","562d27e145cedb3398d6c781"],"authors":[{"id":"5d50c3797390bff0db2a5114","name":"Dor Atzmon"},{"id":"542be28cdabfae216e61cab1","name":"Jiaoyang Li"},{"id":"53f38139dabfae4b349f8a2b","name":"Ariel Felner"},{"id":"53f58dc0dabfaed67df8067c","name":"Eliran Nachmani"},{"id":"53f46741dabfaee4dc851a50","name":"Shahaf S. Shperberg"},{"id":"53f43267dabfaeb1a7bcb584","name":"Nathan R. Sturtevant"},{"id":"562d27e145cedb3398d6c781","name":"Sven Koenig"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/562","headline":"We introduced the multi-directional search algorithm Meet in the Middle* that optimally solves Multi-Agent Meeting problem","id":"5ef96b048806af6ef2772208","is_like":"false","mrt_link":"5f4e0509d463152332381779","num_citation":1,"num_like":0,"num_viewed":130,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0562.pdf","picture":"https://cloud-api.scholarcy.com/images/0562.pdf_olm0ktwd_images_edq1rnau/img-001.png","title":"Multi-Directional Heuristic Search","year":2020},{"aids":["542e1ce9dabfae11fc4c5dd0","53f4399fdabfaee2a1d04811","561d1c8b45cedb339803f3f2","53f42b4adabfaeb22f3e6b4a","560cd53c45cedb339757a831","540543afdabfae91d3fcd837","540683ccdabfae44f083b82c"],"authors":[{"id":"542e1ce9dabfae11fc4c5dd0","name":"Changbin Shao"},{"id":"53f4399fdabfaee2a1d04811","name":"Jing Huo"},{"id":"561d1c8b45cedb339803f3f2","name":"Lei Qi"},{"id":"53f42b4adabfaeb22f3e6b4a","name":"Zhen-Hua Feng"},{"id":"560cd53c45cedb339757a831","name":"Wenbin Li"},{"id":"540543afdabfae91d3fcd837","name":"Chuanqi Dong"},{"id":"540683ccdabfae44f083b82c","name":"Yang Gao"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/93","headline":"The experimental results demonstrated the merits of our Biased Feature Learning method as well as its generalization capability with different network architectures and loss functions","id":"5ef96b048806af6ef2772017","is_like":"false","mrt_link":"5f4e0505d463152332381702","num_citation":0,"num_like":0,"num_viewed":127,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0093.pdf","picture":"https://cloud-api.scholarcy.com/images/0093.pdf_xb8wik8h_images_hyj4wfzq/img-001.png","title":"Biased Feature Learning for Occlusion Invariant Face Recognition","year":2020},{"abstract":"  Unsupervised image captioning with no annotations is an emerging challenge in computer vision, where the existing arts usually adopt GAN (Generative Adversarial Networks) models. In this paper, we propose a novel memory-based network rather than GAN, named Recurrent Relational Memory Network ($R^2M$). Unlike complicated and sensitive adversarial learning that non-ideally performs for long sentence generation, $R^2M$ implements a concepts-to-sentence memory translator through two-stage memory mechanisms: fusion and recurrent memories, correlating the relational reasoning between common visual concepts and the generated words for long periods. $R^2M$ encodes visual context through unsupervised training on images, while enabling the memory to learn from irrelevant textual corpus via supervised fashion. Our solution enjoys less learnable parameters and higher computational efficiency than GAN-based methods, which heavily bear parameter sensitivity. We experimentally validate the superiority of $R^2M$ than state-of-the-arts on all benchmark datasets. ","aids":["542c00fbdabfae2b4e1cbbb2","560dadd545cedb3397609cf0","56cb1891c35f4f3c6565176f"],"authors":[{"id":"542c00fbdabfae2b4e1cbbb2","name":"Dan Guo"},{"id":"560dadd545cedb3397609cf0","name":"Yang Wang"},{"name":"Peipei Song"},{"id":"56cb1891c35f4f3c6565176f","name":"Meng Wang"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/128","headline":"This paper proposes a novel recurrent relational memory network for unsupervised image captioning with low cost of supervision","id":"5ef476b691e01165a63bbbdc","is_like":"false","mrt_link":"5f4e0501d463152332381695","num_citation":0,"num_like":1,"num_viewed":126,"pdf":"https://static.aminer.cn/upload/pdf/884/764/28/5ef476b691e01165a63bbbdc_0.pdf","picture":"https://cloud-api.scholarcy.com/images/5ef476b691e01165a63bbbdc_0.pdf_4du9k826_images_u4qf5m3m/img-001.png","title":"Recurrent Relational Memory Network for Unsupervised Image Captioning","year":2020},{"abstract":"  A major challenge in inductive logic programming (ILP) is learning large programs. We argue that a key limitation of existing systems is that they use entailment to guide the hypothesis search. This approach is limited because entailment is a binary decision: a hypothesis either entails an example or does not, and there is no intermediate position. To address this limitation, we go beyond entailment and use \\emph{example-dependent} loss functions to guide the search, where a hypothesis can partially cover an example. We implement our idea in Brute, a new ILP system which uses best-first search, guided by an example-dependent loss function, to incrementally build programs. Our experiments on three diverse program synthesis domains (robot planning, string transformations, and ASCII art), show that Brute can substantially outperform existing ILP systems, both in terms of predictive accuracies and learning times, and can learn programs 20 times larger than state-of-the-art systems. ","aids":["5d50c38b7390bff0db2a607a","5d50c38f7390bff0db2a63f9"],"authors":[{"id":"5d50c38b7390bff0db2a607a","name":"Cropper Andrew"},{"id":"5d50c38f7390bff0db2a63f9","name":"Dumančić Sebastijan"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/287","headline":"We implemented our idea in Brute, a new inductive logic programming which first invents a library of predicates, including recursive predicates, and performs a best-first search informed by a given loss function to build a hypothesis using the library predicates","id":"5ea16b2b91e011fa08b8f8a2","is_like":"false","mrt_link":"5f4e050ed463152332381804","num_citation":1,"num_like":0,"num_viewed":126,"pdf":"https://static.aminer.cn/storage/pdf/arxiv/20/2004/2004.09855.pdf","picture":"https://cloud-api.scholarcy.com/images/2004.09855.pdf_tz3eo8xy_images_my_y_4ri/img-001.png","title":"Learning large logic programs by going beyond entailment","year":2020},{"aids":["542ead9adabfae4a9e494982","53f447cbdabfaedf435d1f03","53f466c9dabfaeb22f53efd7","53f45084dabfaeecd69d4b63","53f4926bdabfaedd74eb491b"],"authors":[{"id":"542ead9adabfae4a9e494982","name":"Saba Ahmadi"},{"id":"53f447cbdabfaedf435d1f03","name":"Faez Ahmed"},{"id":"53f466c9dabfaeb22f53efd7","name":"John P. Dickerson"},{"id":"53f45084dabfaeecd69d4b63","name":"Mark Fuge"},{"id":"53f4926bdabfaedd74eb491b","name":"Samir Khuller"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/1","headline":"We propose an algorithm that guarantees an optimal solution and converges faster than a proposed approach using a black-box industrial Mixed-Integer Quadratic Program solver","id":"5ef96b048806af6ef2771fd5","is_like":"false","mrt_link":"5f4e0509d46315233238176e","num_citation":1,"num_like":0,"num_viewed":126,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0001.pdf","picture":"https://cloud-api.scholarcy.com/images/0001.pdf_kin01g3p_images_ydd11zeb/img-001.png","title":"An Algorithm for Multi-Attribute Diverse Matching","year":2020},{"aids":["56311fc245cedb3399cb3327","53f3588fdabfae4b34965da0"],"authors":[{"id":"56311fc245cedb3399cb3327","name":"Siyu Lin"},{"id":"53f3588fdabfae4b34965da0","name":"Peter A. Beling"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/627","headline":"We propose an end-to-end optimal trade execution framework based on PPO","id":"5ef96b048806af6ef2772245","is_like":"false","mrt_link":"5f4e0517d4631523323818e3","num_citation":0,"num_like":0,"num_viewed":123,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0627.pdf","picture":"https://cloud-api.scholarcy.com/images/0627.pdf_72fuenkj_images_uqbq8fui/img-001.png","title":"An End-to-End Optimal Trade Execution Framework based on Proximal Policy Optimization","year":2020},{"abstract":"  Existing convergence analyses of Q-learning mostly focus on the vanilla stochastic gradient descent (SGD) type of updates. Despite the Adaptive Moment Estimation (Adam) has been commonly used for practical Q-learning algorithms, there has not been any convergence guarantee provided for Q-learning with such type of updates. In this paper, we first characterize the convergence rate for Q-AMSGrad, which is the Q-learning algorithm with AMSGrad update (a commonly adopted alternative of Adam for theoretical analysis). To further improve the performance, we propose to incorporate the momentum restart scheme to Q-AMSGrad, resulting in the so-called Q-AMSGradR algorithm. The convergence rate of Q-AMSGradR is also established. Our experiments on a linear quadratic regulator problem show that the two proposed Q-learning algorithms outperform the vanilla Q-learning with SGD updates. The two algorithms also exhibit significantly better performance than the DQN learning method over a batch of Atari 2600 games. ","aids":["5617a08e45ce1e5963e405e7","5618326645ce1e5963f93d64","5405a7ecdabfae450f3c22a6","53f4b8f6dabfaed83b77b422"],"authors":[{"id":"5617a08e45ce1e5963e405e7","name":"Bowen Weng"},{"id":"5618326645ce1e5963f93d64","name":"Huaqing Xiong"},{"id":"5405a7ecdabfae450f3c22a6","name":"Yingbin Liang"},{"id":"53f4b8f6dabfaed83b77b422","name":"Wei Zhang"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/422","headline":"We study two Q-learning algorithms with Adaptive Moment Estimation-type updates, and demonstrate their superior performance over the vanilla Q-learning and deep Q-Network algorithms through a linear quadratic regulator problem and a batch of 23 Atari 2600 games","id":"5ef96b048806af6ef27720e4","is_like":"false","mrt_link":"5f4e050ed463152332381805","num_citation":1,"num_like":0,"num_viewed":123,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0422.pdf","picture":"https://cloud-api.scholarcy.com/images/0422.pdf_ikn8agkw_images_86w4v40v/img-001.png","title":"Analysis of Q-learning with Adaptation and Momentum Restart for Gradient Descent","year":2020},{"aids":["53f35095dabfae4b34945bf1","5611f65745ce1e5962a51665","54875be8dabfae9b40134539"],"authors":[{"id":"53f35095dabfae4b34945bf1","name":"Andreas Niskanen"},{"id":"5611f65745ce1e5962a51665","name":"Daniel Neugebauer"},{"id":"54875be8dabfae9b40134539","name":"Matti JÃ¤rvisalo"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/257","headline":"We provided complexity results for credulous and skeptical controllability in control argumentation frameworks under five central semantics","id":"5ef96b048806af6ef27720b8","is_like":"false","mrt_link":"5f4e050ed463152332381810","num_citation":1,"num_like":0,"num_viewed":121,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0257.pdf","picture":"https://cloud-api.scholarcy.com/images/0257.pdf_5chbcjeh_images_uc88qivs/img-001.png","title":"Controllability of Control Argumentation Frameworks","year":2020}],"meta":{"context":"3596084E","time":"811.139271ms"},"succeed":"true","total":645}]}
type(page3)

dict

In [55]:
page2 = {"data":[{"items":[{"aids":["5d50c3b57390bff0db2a8363","562d490f45cedb3398db09bc","53f564e9dabfae60b4f8045b","5630f06645ceb49c5de19c07","53f43bcedabfaec09f1b03a8"],"authors":[{"id":"5d50c3b57390bff0db2a8363","name":"Zhongyang Li"},{"id":"562d490f45cedb3398db09bc","name":"Xiao Ding"},{"id":"53f564e9dabfae60b4f8045b","name":"Ting Liu"},{"id":"5630f06645ceb49c5de19c07","name":"J. Edward Hu"},{"id":"53f43bcedabfaec09f1b03a8","name":"Benjamin Van Durme"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/502","headline":"We introduced a novel extension to lexically-constrained decoding that supports disjunctive positive constraints, where generated output is forced to contain one of a set of candidates","id":"5ef96b048806af6ef27721d9","is_like":"false","mrt_link":"5f4e0503d4631523323816bf","num_citation":1,"num_like":0,"num_viewed":209,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0502.pdf","picture":"https://cloud-api.scholarcy.com/images/0502.pdf_l5v4uks8_images_nivotp40/img-001.png","title":"Guided Generation of Cause and Effect","year":2020},{"abstract":"  Despite the huge progress in scene graph generation in recent years, its long-tail distribution in object relationships remains a challenging and pestering issue. Existing methods largely rely on either external knowledge or statistical bias information to alleviate this problem. In this paper, we tackle this issue from another two aspects: (1) scene-object interaction aiming at learning specific knowledge from a scene via an additive attention mechanism; and (2) long-tail knowledge transfer which tries to transfer the rich knowledge learned from the head into the tail. Extensive experiments on the benchmark dataset Visual Genome on three tasks demonstrate that our method outperforms current state-of-the-art competitors. ","aids":["53f42e9cdabfaeb2acffa044","53f44ba9dabfaeecd69c1ed7","53f438f4dabfaee43ec48960","542e56fadabfae2222dd1313"],"authors":[{"name":"He Tao"},{"id":"53f42e9cdabfaeb2acffa044","name":"Gao Lianli"},{"id":"53f44ba9dabfaeecd69c1ed7","name":"Song Jingkuan"},{"id":"53f438f4dabfaee43ec48960","name":"Cai Jianfei"},{"id":"542e56fadabfae2222dd1313","name":"Li Yuan-Fang"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/82","headline":"We investigate the long-tail problem existing in scene graph generation","id":"5ee8986891e011e66831c322","is_like":"false","mrt_link":"5f4e0501d46315233238168e","num_citation":0,"num_like":0,"num_viewed":205,"pdf":"https://static.aminer.cn/upload/pdf/209/1152/629/5ee8986891e011e66831c322_0.pdf","picture":"https://cloud-api.scholarcy.com/images/5ee8986891e011e66831c322_0.pdf_8n3tc_9z_images_2w9g6ph0/img-001.png","title":"Learning from the Scene and Borrowing from the Rich: Tackling the Long\n  Tail in Scene Graph Generation","year":2020},{"abstract":"  Generative adversarial imitation learning (GAIL) has shown promising results by taking advantage of generative adversarial nets, especially in the field of robot learning. However, the requirement of isolated single modal demonstrations limits the scalability of the approach to real world scenarios such as autonomous vehicles' demand for a proper understanding of human drivers' behavior. In this paper, we propose a novel multi-modal GAIL framework, named Triple-GAIL, that is able to learn skill selection and imitation jointly from both expert demonstrations and continuously generated experiences with data augmentation purpose by introducing an auxiliary skill selector. We provide theoretical guarantees on the convergence to optima for both of the generator and the selector respectively. Experiments on real driver trajectories and real-time strategy game datasets demonstrate that Triple-GAIL can better fit multi-modal behaviors close to the demonstrators and outperforms state-of-the-art methods. ","aids":["5632006045cedb3399f8cbab","53f43454dabfaedce551b305","53f4327bdabfaeb22f449275","53f43a9cdabfaeb2ac075577"],"authors":[{"name":"Fei Cong"},{"name":"Wang Bin"},{"name":"Zhuang Yuzheng"},{"id":"5632006045cedb3399f8cbab","name":"Zhang Zongzhang"},{"id":"53f43454dabfaedce551b305","name":"Hao Jianye"},{"name":"Zhang Hongbo"},{"id":"53f4327bdabfaeb22f449275","name":"Ji Xuewu"},{"id":"53f43a9cdabfaeb2ac075577","name":"Liu Wulong"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/405","headline":"We propose a novel multi-modal GAIL framework, named Triple-GAIL, that is able to learn skill selection and imitation jointly from both expert demonstrations and continuously generated experiences with data augmentation purpose by introducing an auxiliary skill selector","id":"5ec7a33091e0118397f3ef17","is_like":"false","mrt_link":"5f4e0502d4631523323816a7","num_citation":0,"num_like":1,"num_viewed":203,"pdf":"https://static.aminer.cn/storage/pdf/arxiv/20/2005/2005.10622.pdf","picture":"https://cloud-api.scholarcy.com/images/2005.10622.pdf_ic74czz6_images_lt7brm5_/img-001.png","title":"Triple-GAIL: A Multi-Modal Imitation Learning Framework with Generative\n  Adversarial Nets","year":2020},{"abstract":"  A feature-based model explanation denotes how much each input feature contributes to a model\'s output for a given data point. As the number of proposed explanation functions grows, we lack quantitative evaluation criteria to help practitioners know when to use which explanation function. This paper proposes quantitative evaluation criteria for feature-based explanations: low sensitivity, high faithfulness, and low complexity. We devise a framework for aggregating explanation functions. We develop a procedure for learning an aggregate explanation function with lower complexity and then derive a new aggregate Shapley value explanation function that minimizes sensitivity. ","aids":["5628b44445ce1e596600e0eb","53f4325cdabfaedce5504bfa","548c9bc1dabfaed7b5fa44fa"],"authors":[{"id":"5628b44445ce1e596600e0eb","name":"Bhatt Umang"},{"id":"53f4325cdabfaedce5504bfa","name":"Weller Adrian"},{"id":"548c9bc1dabfaed7b5fa44fa","name":"Moura José M. F."}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/417","headline":"This paper proposes quantitative evaluation criteria for feature-based explanations: low sensitivity, high faithfulness, and low complexity","id":"5eb78919da5629cf244303c0","is_like":"false","mrt_link":"5f4e0509d46315233238177d","num_citation":4,"num_like":0,"num_viewed":200,"pdf":"https://static.aminer.cn/storage/pdf/arxiv/20/2005/2005.00631.pdf","picture":"https://cloud-api.scholarcy.com/images/2005.00631.pdf_j0bhk9ny_images_4amnsm6z/img-001.png","title":"Evaluating and Aggregating Feature-based Model Explanations","year":2020},{"aids":["53f45d1ddabfaec09f21d5d5","53f43759dabfaee1c0aa5d40","53f42c26dabfaeb22f3f5982","53f49743dabfaeb6cf77b539"],"authors":[{"id":"53f45d1ddabfaec09f21d5d5","name":"Bahare Fatemi"},{"id":"53f43759dabfaee1c0aa5d40","name":"Perouz Taslakian"},{"id":"53f42c26dabfaeb22f3f5982","name":"David Vazquez"},{"id":"53f49743dabfaeb6cf77b539","name":"David Poole"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/303","headline":"Given a knowledge hypergraph on τ, we evaluate various completion methods using a train and test set τtrain and τtest","id":"5ef96b048806af6ef2772134","is_like":"false","mrt_link":"5f4e0512d46315233238185c","num_citation":9,"num_like":0,"num_viewed":194,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0303.pdf","picture":"https://cloud-api.scholarcy.com/images/0303.pdf_jj5_7bb__images_rndq2j_x/img-001.png","title":"Knowledge Hypergraphs: Prediction Beyond Binary Relations","year":2020},{"abstract":"  Reinforcement learning agents usually learn from scratch, which requires a large number of interactions with the environment. This is quite different from the learning process of human. When faced with a new task, human naturally have the common sense and use the prior knowledge to derive an initial policy and guide the learning process afterwards. Although the prior knowledge may be not fully applicable to the new task, the learning process is significantly sped up since the initial policy ensures a quick-start of learning and intermediate guidance allows to avoid unnecessary exploration. Taking this inspiration, we propose knowledge guided policy network (KoGuN), a novel framework that combines human prior suboptimal knowledge with reinforcement learning. Our framework consists of a fuzzy rule controller to represent human knowledge and a refine module to fine-tune suboptimal prior knowledge. The proposed framework is end-to-end and can be combined with existing policy-based reinforcement learning algorithm. We conduct experiments on both discrete and continuous control tasks. The empirical results show that our approach, which combines human suboptimal knowledge and RL, achieves significant improvement on learning efficiency of flat RL algorithms, even with very low-performance human prior knowledge. ","aids":["53f43454dabfaedce551b305","560fe71345ce1e5961f0fe0f","53f443d2dabfaeb22f4b53ba","562d3e7745cedb3398d9c324"],"authors":[{"name":"Zhang Peng"},{"id":"53f43454dabfaedce551b305","name":"Hao Jianye"},{"id":"560fe71345ce1e5961f0fe0f","name":"Wang Weixun"},{"id":"53f443d2dabfaeb22f4b53ba","name":"Tang Hongyao"},{"name":"Ma Yi"},{"id":"562d3e7745cedb3398d9c324","name":"Duan Yihai"},{"name":"Zheng Yan"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/317","headline":"We propose a novel policy network framework called knowledge guided policy network to leverage human knowledge to accelerate the learning process of RL agents","id":"5e4d083f3a55ac8cfd770c23","is_like":"false","mrt_link":"5f4e0502d463152332381699","num_citation":0,"num_like":1,"num_viewed":188,"pdf":"https://static.aminer.cn/storage/pdf/arxiv/20/2002/2002.07418.pdf","picture":"https://cloud-api.scholarcy.com/images/2002.07418.pdf_3vpwq_7a_images_hjpmte6a/img-001.png","title":"KoGuN: Accelerating Deep Reinforcement Learning via Integrating Human\n  Suboptimal Knowledge","year":2020},
                           {"aids":["5d415be07390bff0db70a4c7","5d415c0c7390bff0db70c49b","53f478d1dabfaee4dc8965ac","5603964b45cedb33961dffac","542a239edabfae864af907a1","53fa102edabfae7f97b045c2","542d413cdabfae11fc4517d5"],"authors":[{"id":"5d415be07390bff0db70a4c7","name":"Renjun Hu"},{"id":"5d415c0c7390bff0db70c49b","name":"Xinjiang Lu"},{"id":"53f478d1dabfaee4dc8965ac","name":"Chuanren Liu"},{"id":"5603964b45cedb33961dffac","name":"Yanyan Li"},{"id":"542a239edabfae864af907a1","name":"Hao Liu"},{"id":"53fa102edabfae7f97b045c2","name":"Shuai Ma"},{"id":"542d413cdabfae11fc4517d5","name":"Hui Xiong"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/478","headline":"Its relationship to decision profiling is that PROUD preserves decision structures via identifying key factors and maximizing scalar projection, and the effectiveness of PROUD for preserving decision structures directly relies on the goodness of identified key factors","id":"5e5e193693d709897ce5ca82","is_like":"false","mrt_link":"5f4e0509d46315233238177a","num_citation":1,"num_like":0,"num_viewed":188,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0478.pdf","picture":"https://cloud-api.scholarcy.com/images/0478.pdf_g27sfckn_images_8vmag6yz/img-001.png","title":"Why We Go Where We Go: Profiling User Decisions on Choosing POIs","year":2020},{"aids":["5609ae0845cedb3396f4be32","560defcb45cedb3397628128","5d415c0a7390bff0db70c3c3","560763c045cedb3396a9e0c4","562d4b8b45cedb3398db54eb","562d24f745cedb3398d668a9"],"authors":[{"id":"5609ae0845cedb3396f4be32","name":"Zihao Zhu"},{"id":"560defcb45cedb3397628128","name":"Jing Yu"},{"id":"5d415c0a7390bff0db70c3c3","name":"Yujing Wang"},{"id":"560763c045cedb3396a9e0c4","name":"Yajing Sun"},{"id":"562d4b8b45cedb3398db54eb","name":"Yue Hu"},{"id":"562d24f745cedb3398d668a9","name":"Qi Wu"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/153","headline":"We propose Mucko for visual question answering requiring external knowledge, which focuses on multilayer cross-modal knowledge reasoning","id":"5ef96b048806af6ef2772043","is_like":"false","mrt_link":"5f4e0514d463152332381896","num_citation":0,"num_like":0,"num_viewed":181,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0153.pdf","picture":"https://cloud-api.scholarcy.com/images/0153.pdf_mj9hi_3s_images_02ywp8bp/img-001.png","title":"Mucko: Multi-Layer Cross-Modal Knowledge Reasoning for Fact-based Visual Question Answering","year":2020},{"aids":["5628e87545ce1e59660dc3bc","561546a545ce1e59636a2271","5433d1e9dabfaebba5829eaa","5611d59445ce1e59629b58f2","56083a9745ce1e595f4cc503","53f42cb6dabfaeb1a7b81161","53f55f34dabfae535bf8045b"],"authors":[{"id":"5628e87545ce1e59660dc3bc","name":"Riley Simmons-Edler"},{"id":"561546a545ce1e59636a2271","name":"Ben Eisner"},{"id":"5433d1e9dabfaebba5829eaa","name":"Daniel Yang"},{"id":"5611d59445ce1e59629b58f2","name":"Anthony Bisulco"},{"id":"56083a9745ce1e595f4cc503","name":"Eric Mitchell"},{"id":"53f42cb6dabfaeb1a7b81161","name":"Sebastian Seung"},{"id":"53f55f34dabfae535bf8045b","name":"Daniel Lee"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/390","headline":"We propose a new exploration objective, maximizing the reward prediction error of a value function trained to predict extrinsic reward","id":"5ef96b048806af6ef2772170","is_like":"false","mrt_link":"5f4e0515d4631523323818a7","num_citation":2,"num_like":0,"num_viewed":179,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0390.pdf","picture":"https://cloud-api.scholarcy.com/images/0390.pdf_e92q5ln__images_k15nl98_/img-001.png","title":"Reward Prediction Error as an Exploration Objective in Deep RL","year":2020},{"aids":["5629baf345ce1e596650b707","53f58dc0dabfaed67df8060d","560c208445cedb339749f1d4","544098b0dabfae805a6d9817","53f4cb26dabfaeeb13f80965"],"authors":[{"id":"5629baf345ce1e596650b707","name":"Gabriele Ciravegna"},{"id":"53f58dc0dabfaed67df8060d","name":"Francesco Giannini"},{"id":"560c208445cedb339749f1d4","name":"Marco Gori"},{"id":"544098b0dabfae805a6d9817","name":"Marco Maggini"},{"id":"53f4cb26dabfaeeb13f80965","name":"Stefano Melacci"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/309","headline":"We presented an approach that yields First-Order Logicbased explanations of a multi-label neural classifier, using another neural network that learns to explain the classifier itself","id":"5ef96b048806af6ef2772124","is_like":"false","mrt_link":"5f4e050dd4631523323817f0","num_citation":1,"num_like":0,"num_viewed":174,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0309.pdf","picture":"https://cloud-api.scholarcy.com/images/0309.pdf_hqeixdd2_images_qkb56q5z/img-001.png","title":"Human-Driven FOL Explanations of Deep Learning","year":2020},{"abstract":"  Stochastic variance-reduced gradient (SVRG) is an optimization method originally designed for tackling machine learning problems with a finite sum structure. SVRG was later shown to work for policy evaluation, a problem in reinforcement learning in which one aims to estimate the value function of a given policy. SVRG makes use of gradient estimates at two scales. At the slower scale, SVRG computes a full gradient over the whole dataset, which could lead to prohibitive computation costs. In this work, we show that two variants of SVRG for policy evaluation could significantly diminish the number of gradient calculations while preserving a linear convergence speed. More importantly, our theoretical result implies that one does not need to use the entire dataset in every epoch of SVRG when it is applied to policy evaluation with linear function approximation. Our experiments demonstrate large computational savings provided by the proposed methods. ","aids":["562d535845cedb3398dc60f2","53f430c5dabfaedd74d61d58","53f4cb73dabfaee57b78116d","5434eb1cdabfaebba587df4e"],"authors":[{"id":"562d535845cedb3398dc60f2","name":"Zilun Peng"},{"id":"53f430c5dabfaedd74d61d58","name":"Ahmed Touati"},{"id":"53f4cb73dabfaee57b78116d","name":"Pascal Vincent"},{"id":"5434eb1cdabfaebba587df4e","name":"Doina Precup"}],"conf_id":"5ef86d3592c7f9be218d4b9b","headline":"We show that Batching Stochastic variance-reduced gradient and Stochastic Gradient converge linearly when solving the saddle-point formulation of Mean Squared Projected Bellman Error","id":"5d06e491da562926acc53f32","is_like":"false","lang":"en","mrt_link":"5f4e0501d46315233238168f","num_citation":1,"num_like":0,"num_viewed":169,"pdf":"https://static.aminer.cn/storage/pdf/arxiv/19/1906/1906.03704.pdf","picture":"https://cloud-api.scholarcy.com/images/1906.03704.pdf_5jbexv5c_images_0bvvf0la/img-001.png","title":"SVRG for Policy Evaluation with Fewer Gradient Evaluations.","year":2019},{"abstract":"  Estimating probability distribution is one of the core issues in the NLP field. However, in both deep learning (DL) and pre-DL eras, unlike the vast applications of linear-chain CRF in sequence labeling tasks, very few works have applied tree-structure CRF to constituency parsing, mainly due to the complexity and inefficiency of the inside-outside algorithm. This work presents a fast and accurate neural CRF constituency parser. The key idea is to batchify the inside algorithm for loss computation by direct large tensor operations on GPU, and meanwhile avoid the outside algorithm for gradient computation via efficient back-propagation. We also propose a simple two-stage bracketing-then-labeling parsing approach to improve efficiency further. To improve the parsing performance, inspired by recent progress in dependency parsing, we introduce a new scoring architecture based on boundary representation and biaffine attention, and a beneficial dropout strategy. Experiments on PTB, CTB5.1, and CTB7 show that our two-stage CRF parser achieves new state-of-the-art performance on both settings of w/o and w/ BERT, and can parse over 1,000 sentences per second. We release our code at https://github.com/yzhangcs/crfpar. ","aids":["560dbbe345ce1e5960ef7b3d","560caea345cedb339755e6ac","53f46586dabfaee2a1dab376"],"authors":[{"id":"560dbbe345ce1e5960ef7b3d","name":"Yu Zhang"},{"id":"560caea345cedb339755e6ac","name":"Houquan Zhou"},{"id":"53f46586dabfaee2a1dab376","name":"Zhenghua Li"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/560","headline":"We propose a fast and accurate neural CRF constituency parser","id":"5ef96b048806af6ef27721d2","is_like":"false","mrt_link":"5f4e0514d463152332381895","num_citation":0,"num_like":0,"num_viewed":169,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0560.pdf","picture":"https://cloud-api.scholarcy.com/images/0560.pdf_c4bfbcvw_images_4gfmi8wm/img-001.png","title":"Fast and Accurate Neural CRF Constituency Parsing","year":2020},{"abstract":"  Commonsense knowledge acquisition is a key problem for artificial intelligence. Conventional methods of acquiring commonsense knowledge generally require laborious and costly human annotations, which are not feasible on a large scale. In this paper, we explore a practical way of mining commonsense knowledge from linguistic graphs, with the goal of transferring cheap knowledge obtained with linguistic patterns into expensive commonsense knowledge. The result is a conversion of ASER [Zhang et al., 2020], a large-scale selectional preference knowledge resource, into TransOMCS, of the same representation as ConceptNet [Liu and Singh, 2004] but two orders of magnitude larger. Experimental results demonstrate the transferability of linguistic knowledge to commonsense knowledge and the effectiveness of the proposed approach in terms of quantity, novelty, and quality. TransOMCS is publicly available at: https://github.com/HKUST-KnowComp/TransOMCS. ",
                           "aids":["560fe00145ce1e5961ede495","53f42f85dabfaec22ba333b0","54849c88dabfae9b40132fad","53f48b48dabfaee0d9c73c7f"],"authors":[{"id":"560fe00145ce1e5961ede495","name":"Zhang Hongming"},{"id":"53f42f85dabfaec22ba333b0","name":"Khashabi Daniel"},{"id":"54849c88dabfae9b40132fad","name":"Song Yangqiu"},{"id":"53f48b48dabfaee0d9c73c7f","name":"Roth Dan"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/554","headline":"We explore a practical way of mining commonsense knowledge from linguistic graphs, with the goal of transferring cheap knowledge obtained with linguistic patterns into expensive commonsense knowledge","id":"5eafe7e091e01198d39865dc","is_like":"false","mrt_link":"5f4e0501d463152332381692","num_citation":2,"num_like":0,"num_viewed":167,"pdf":"https://static.aminer.cn/storage/pdf/arxiv/20/2005/2005.00206.pdf","picture":"https://cloud-api.scholarcy.com/images/2005.00206.pdf_11fs6_gg_images_5kzbp9hc/img-001.png","title":"TransOMCS: From Linguistic Graphs to Commonsense Knowledge","year":2020},{"aids":["5405fb80dabfae91d3023aaf","5d43d51b7390bff0db5fefda","53f4cdf8dabfaeedd777b4a3","53f43766dabfaee02acd305b","53f431aadabfaee0d9b33395","53f463f5dabfaeb22f532fed"],"authors":[{"id":"5405fb80dabfae91d3023aaf","name":"Jinhuan Liu"},{"id":"5d43d51b7390bff0db5fefda","name":"Xuemeng Song"},{"id":"53f4cdf8dabfaeedd777b4a3","name":"Zhaochun Ren"},{"id":"53f43766dabfaee02acd305b","name":"Liqiang Nie"},{"id":"53f431aadabfaee0d9b33395","name":"Zhaopeng Tu"},{"id":"53f463f5dabfaeb22f532fed","name":"Jun Ma"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/485","headline":"We present an end-to-end auxiliary templateenhanced generative compatibility modeling scheme, which is able to comprehensively model the compatibility between fashion items from both the item-item and item-template perspectives","id":"5ef96b048806af6ef27721a8","is_like":"false","mrt_link":"5f4e0501d463152332381688","num_citation":0,"num_like":0,"num_viewed":166,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0485.pdf","picture":"https://cloud-api.scholarcy.com/images/0485.pdf_p2bf22z__images_pqfj2wt8/img-001.png","title":"Auxiliary Template-Enhanced Generative Compatibility Modeling","year":2020},{"abstract":"  Vehicle re-identification (reID) aims at identifying vehicles across different non-overlapping cameras views. The existing methods heavily relied on well-labeled datasets for ideal performance, which inevitably causes fateful drop due to the severe domain bias between the training domain and the real-world scenes; worse still, these approaches required full annotations, which is labor-consuming. To tackle these challenges, we propose a novel progressive adaptation learning method for vehicle reID, named PAL, which infers from the abundant data without annotations. For PAL, a data adaptation module is employed for source domain, which generates the images with similar data distribution to unlabeled target domain as ``pseudo target samples\'\'. These pseudo samples are combined with the unlabeled samples that are selected by a dynamic sampling strategy to make training faster. We further proposed a weighted label smoothing (WLS) loss, which considers the similarity between samples with different clusters to balance the confidence of pseudo labels. Comprehensive experimental results validate the advantages of PAL on both VehicleID and VeRi-776 dataset. ","aids":["560dadd545cedb3397609cf0","53f42ff2dabfaeb2ac00a1d6","57198d7776d911115b5dc3bd","542dd861dabfae11fc4a7bf6","56cb1891c35f4f3c6565176f"],"authors":[{"name":"Jinjia Peng"},{"id":"560dadd545cedb3397609cf0","name":"Yang Wang"},{"id":"53f42ff2dabfaeb2ac00a1d6","name":"Huibing Wang"},{"id":"57198d7776d911115b5dc3bd","name":"Zhao Zhang"},{"id":"542dd861dabfae11fc4a7bf6","name":"Xianping Fu"},{"id":"56cb1891c35f4f3c6565176f","name":"Meng Wang"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/127","headline":"We propose an unsupervised vehicle reID framework, named PAL, which iteratively updates the feature learning model and estimates pseudo labels for unlabeled data for target domain adaptation","id":"5ef3247091e0110c353da56d","is_like":"false","mrt_link":"5f4e0501d46315233238168b","num_citation":0,"num_like":1,"num_viewed":166,"pdf":"https://static.aminer.cn/upload/pdf/1717/1911/1132/5ef3247091e0110c353da56d_0.pdf","picture":"https://cloud-api.scholarcy.com/images/5ef3247091e0110c353da56d_0.pdf_z638jc68_images_4xklfetw/img-001.png","title":"Unsupervised Vehicle Re-identification with Progressive Adaptation","year":2020},{"abstract":"  In this paper, we show that a simple coloring scheme can improve, both theoretically and empirically, the expressive power of Message Passing Neural Networks(MPNNs). More specifically, we introduce a graph neural network called Colored Local Iterative Procedure (CLIP) that uses colors to disambiguate identical node attributes, and show that this representation is a universal approximator of continuous functions on graphs with node attributes. Our method relies on separability , a key topological characteristic that allows to extend well-chosen neural networks into universal representations. Finally, we show experimentally that CLIP is capable of capturing structural characteristics that traditional MPNNs fail to distinguish,while being state-of-the-art on benchmark graph classification datasets. ","aids":["53f35d3edabfae4b34977070","562c7b2c45cedb3398c35dd5"],"authors":[{"id":"53f35d3edabfae4b34977070","name":"Dasoulas George"},{"name":"Santos Ludovic Dos"},{"id":"562c7b2c45cedb3398c35dd5","name":"Scaman Kevin"},{"name":"Virmaux Aladin"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/294","headline":"Universality was proven using the novel concept of separable neural networks, and our experiments showed that Colored Local Iterative Procedure is state-of-the-art on both graph classification datasets and property testing tasks","id":"5df371de3a55acfd20674ba8","is_like":"false","mrt_link":"5f4e050dd4631523323817dd","num_citation":3,"num_like":0,"num_viewed":165,"pdf":"https://static.aminer.cn/upload/pdf/466/25/441/5df371de3a55acfd20674ba8_0.pdf","picture":"https://cloud-api.scholarcy.com/images/5df371de3a55acfd20674ba8_0.pdf_ug7307ym_images_9e96dipr/img-001.png","title":"Coloring graph neural networks for node disambiguation","year":2019},{"abstract":"  We propose a simple yet powerful test statistic to quantify the discrepancy between two conditional distributions. The new statistic avoids the explicit estimation of the underlying distributions in highdimensional space and it operates on the cone of symmetric positive semidefinite (SPS) matrix using the Bregman matrix divergence. Moreover, it inherits the merits of the correntropy function to explicitly incorporate high-order statistics in the data. We present the properties of our new statistic and illustrate its connections to prior art. We finally show the applications of our new statistic on three different machine learning problems, namely the multi-task learning over graphs, the concept drift detection, and the information-theoretic feature selection, to demonstrate its utility and advantage. Code of our statistic is available at https://bit.ly/BregmanCorrentropy. ","aids":["562d86fd45cedb3398e45480","562ccd5945cedb3398ccb50a","562c73fc45cedb3398c289d7","53f46c52dabfaee2a1dc6835"],"authors":[{"id":"562d86fd45cedb3398e45480","name":"Yu Shujian"},{"id":"562ccd5945cedb3398ccb50a","name":"Shaker Ammar"},{"id":"562c73fc45cedb3398c289d7","name":"Alesiani Francesco"},{"id":"53f46c52dabfaee2a1dc6835","name":"Principe Jose C."}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/385","headline":"Our statistic enables the development of alternative solutions to classical machine learning problems in a fundamental manner","id":"5eb3dc4191e011ce31f280e8","is_like":"false","mrt_link":"5f4e0506d46315233238171b","num_citation":1,"num_like":0,"num_viewed":163,"pdf":"https://static.aminer.cn/storage/pdf/arxiv/20/2005/2005.02196.pdf","picture":"https://cloud-api.scholarcy.com/images/2005.02196.pdf_dvoxcls2_images_f1x30x8n/img-001.png","title":"Measuring the Discrepancy between Conditional Distributions: Methods,\n  Properties and Applications","year":2020},{"aids":["542a4c5fdabfae646d54a975","5630fa8645ceb49c5de51fe2","53f48bdddabfaea7cd1cd3b6","53f58dbfdabfaed67df80495","5429ae0fdabfaec7081a5ec0"],"authors":[{"id":"542a4c5fdabfae646d54a975","name":"Min Shi"},{"id":"5630fa8645ceb49c5de51fe2","name":"Yufei Tang"},{"id":"53f48bdddabfaea7cd1cd3b6","name":"Xingquan Zhu"},{"id":"53f58dbfdabfaed67df80495","name":"David Wilson"},{"id":"5429ae0fdabfaec7081a5ec0","name":"Jianxun Liu"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/398","headline":"We proposed a novel dual-regularized graph convolutional network that contains a conditional adversarial training to enhance the separation of nodes from different classes and a distribution alignment training to enforce balanced learning between majority and minority classes","id":"5ef96b048806af6ef277214e","is_like":"false","mrt_link":"5f4e0509d46315233238176f","num_citation":0,"num_like":0,"num_viewed":162,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0398.pdf","picture":"https://cloud-api.scholarcy.com/images/0398.pdf_8x4vvwz5_images_8sfuhlb0/img-001.png","title":"Multi-Class Imbalanced Graph Convolutional Network Learning","year":2020},{"abstract":"  RGB-Infrared (IR) cross-modality person re-identification (re-ID), which aims to search an IR image in RGB gallery or vice versa, is a challenging task due to the large discrepancy between IR and RGB modalities. Existing methods address this challenge typically by aligning feature distributions or image styles across modalities, whereas the very useful similarities among gallery samples of the same modality (i.e. intra-modality sample similarities) is largely neglected. This paper presents a novel similarity inference metric (SIM) that exploits the intra-modality sample similarities to circumvent the cross-modality discrepancy targeting optimal cross-modality image matching. SIM works by successive similarity graph reasoning and mutual nearest-neighbor reasoning that mine cross-modality sample similarities by leveraging intra-modality sample similarities from two different perspectives. Extensive experiments over two cross-modality re-ID datasets (SYSU-MM01 and RegDB) show that SIM achieves significant accuracy improvement but with little extra training as compared with the state-of-the-art. ",
                           "aids":["53f449fcdabfaee1c0afb982","560bc4a445ce1e596038379f","5405b433dabfae92b41f65e9","54408ee3dabfae7d84b81202","542d4074dabfae11fc4511e0"],"authors":[{"id":"53f449fcdabfaee1c0afb982","name":"Mengxi Jia"},{"id":"560bc4a445ce1e596038379f","name":"Yunpeng Zhai"},{"id":"5405b433dabfae92b41f65e9","name":"Shijian Lu"},{"id":"54408ee3dabfae7d84b81202","name":"Siwei Ma"},{"id":"542d4074dabfae11fc4511e0","name":"Jian Zhang"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/143","headline":"We introduce similarity graph reasoning and mutual nearestneighbor reasoning to infer inter-modality sample similarities by exploiting reliable intra-modality sample similarity","id":"5ef96b048806af6ef2772012","is_like":"false","mrt_link":"5f4e050dd4631523323817d8","num_citation":1,"num_like":0,"num_viewed":161,"pdf":"https://static.aminer.cn/storage/pdf/arxiv/20/2007/2007.01504.pdf","picture":"https://cloud-api.scholarcy.com/images/2007.01504.pdf_hlyyqkux_images_88i7wr7a/img-001.png","title":"A Similarity Inference Metric for RGB-Infrared Cross-Modality Person Re-identification","year":2020},{"aids":["542b6ab0dabfae2b4e163eba","560527d345ce1e595e41039f","53f3852edabfae4b34a0e3c2","5614156c45ce1e596330dbc3","56065cde45cedb3396854f9a"],"authors":[{"id":"542b6ab0dabfae2b4e163eba","name":"Weiwei Wang"},{"id":"560527d345ce1e595e41039f","name":"Yuming Shen"},{"id":"53f3852edabfae4b34a0e3c2","name":"Haofeng Zhang"},{"id":"5614156c45ce1e596330dbc3","name":"Yazhou Yao"},{"id":"56065cde45cedb3396854f9a","name":"Li Liu"}],"conf_id":"5ef86d3592c7f9be218d4b9b","doi":"10.24963/ijcai.2020/119","headline":"This paper proposed a new kind of unsupervised crossmodal hashing method which utilized sparse graph structures to exploit similarity information to address the degradation problem in unsupervised algorithms","id":"5ef96b048806af6ef277205a","is_like":"false","mrt_link":"5f4e050bd4631523323817b2","num_citation":2,"num_like":0,"num_viewed":159,"pdf":"https://static.aminer.cn/storage/pdf/ijcai/20/0119.pdf","picture":"https://cloud-api.scholarcy.com/images/0119.pdf_219vsh2h_images_pg2dql84/img-001.png","title":"Set and Rebase: Determining the Semantic Graph Connectivity for Unsupervised Cross-Modal Hashing","year":2020}],"meta":{"context":"E1E67B45","time":"819.293121ms"},"succeed":"true","total":645}]}


In [56]:
type(page2)

dict

In [93]:
folder_location = r'D:\Ani\Thesis\papers'
for paper in page3['data'][0]['items']:
    title = paper['title'].replace('\n', '').replace(':', '')
    filename = os.path.join(folder_location, title + '.pdf')
    with open(filename, 'wb') as f:
        f.write(requests.get(paper['pdf']).content)

In [92]:
for i in page2['data'][0]['items']:
    print(i['title'], '\n')

Guided Generation of Cause and Effect 

Learning from the Scene and Borrowing from the Rich: Tackling the Long
  Tail in Scene Graph Generation 

Triple-GAIL: A Multi-Modal Imitation Learning Framework with Generative
  Adversarial Nets 

Evaluating and Aggregating Feature-based Model Explanations 

Knowledge Hypergraphs: Prediction Beyond Binary Relations 

KoGuN: Accelerating Deep Reinforcement Learning via Integrating Human
  Suboptimal Knowledge 

Why We Go Where We Go: Profiling User Decisions on Choosing POIs 

Mucko: Multi-Layer Cross-Modal Knowledge Reasoning for Fact-based Visual Question Answering 

Reward Prediction Error as an Exploration Objective in Deep RL 

Human-Driven FOL Explanations of Deep Learning 

SVRG for Policy Evaluation with Fewer Gradient Evaluations. 

Fast and Accurate Neural CRF Constituency Parsing 

TransOMCS: From Linguistic Graphs to Commonsense Knowledge 

Auxiliary Template-Enhanced Generative Compatibility Modeling 

Unsupervised Vehicle Re-identi

In [37]:
url_search = 'https://apiv2.aminer.cn/magic?a=GetPubs__conf.GetPubs___'

In [31]:
headers={"Accept": "application/json",
         "Accept-Encoding": "gzip, deflate, br",
         "Accept-Language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7,hy;q=0.6",
         "Authorization": "eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJzdWIiOiIyLURvUm1kdVwvY0F2VUxMNzhiMDVyY1FaNElrUWlGTmJuRk1JYUdaNEllQ0YwM0JhdExQbWkyWnJmbGlUZzVkcUFDczhlNGwrNTJDcllIYWF4ZGdnckxCb0VHRkljNXkyMnVXcmpcLzc2MndnNVE9IiwidWlkIjoiNWZiYThmMzM0Yzc3NWU1YTQ3MTcwYmM5Iiwic3JjIjoiYW1pbmVyIiwicm9sZXMiOltdLCJpc3MiOiJhcGkuYW1pbmVyLm9yZyIsImV4cCI6MTYwODY1NDAwMCwiaWF0IjoxNjA2MDYyMDAwLCJqdGkiOiI4OWI3YTA4YzEwM2I3NWY5NDNkZjNiMjlmYTJhNjI4ZDQyMmE1M2IwNTI2NjI3ZmNlYmIzZWFhMWU2MmFhNjNlYTJhMDcyMmU4NmZhNDRjZGU4NDg4MDZlNmVmOGQxYjI5MmYxYTk4NDExOWRiOGI1NTc4YzJkMmE1ZDY0M2ZjMTg4NmM4MTYzZmY1NzhhZTE2ZjM0ZWI0NDc0MGU4OWIzMjI2Y2ZjZWRmZTcxZWRjM2I0MWIyOTgyZDEyZWIwNzY1NWQxMGU5YWVjNzdhZmJmMGUwNWMzYTIzMGEyYmQ2MWY3ZTZjN2E2MTY1OGE3NTllMDA3NzUyMDE0YzliMzJiIiwiZW1haWwiOiJ0ZXZvc3lhbmFuaUBtYWlsLnJ1In0.Lk55y9j1N6B2sfacWRv6HbW3N5w74_rxBfJdvvQBosw",
         "Connection": "keep-alive",
         "Content-Length": "345",
         "Content-Type": "application/json",
         "debug": "1",
         "Host": "apiv2.aminer.cn",
         "Origin": "https://www.aminer.org",
         "Referer": "https://www.aminer.org/",
         "sec-ch-ua": "'Google Chrome';v='87', '\'Not;A\\Brand';v='99', 'Chromium';v='87'",
         "sec-ch-ua-mobile": "?1",
         "Sec-Fetch-Dest": "empty",
         "Sec-Fetch-Mode": "cors",
         "Sec-Fetch-Site": "cross-site",
         "User-Agent":"Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Mobile Safari/537.36"}

data={
    "action": "conf.GetPubs",
    "parameters": {"conf_id": "5ef86d3592c7f9be218d4b9b", "offset": 20, "size": 20, "sort": "view"},
    "schema":
        {"publication": ["id", "year", "title", "title_zh", "abstract", "abstract_zh", "authors", "authors._id"]}
    }

In [38]:
req=requests.post(url_search,headers=headers,json=data)

In [18]:
type(req)

requests.models.Response

In [39]:
req.text

'error on decode request: json: cannot unmarshal object into Go value of type []webmodel.RequestData\n'

## American Journal of Mathematical and Computer Modelling

In [12]:
vol_issue = [(1, 1),
             (2, 1), (2, 2), (2, 3), (2, 4),
             (3, 1), (3, 2), (3, 3), (3, 4),
             (4, 1), (4, 2), (4, 3), (4, 4),
             (5, 1), (5, 2), (5, 3), (5, 4)]

url_base = 'http://www.sciencepublishinggroup.com/journal/archive?journalid=604&issueid=604'

In [13]:
folder_location = r'D:\Ani\Thesis\sciencePG\Machine Learning Research'
for i in tqdm(vol_issue):
    url = url_base + '0' + str(i[0]) + '0' + str(i[1])
    response = requests.get(url)
    soup= BeautifulSoup(response.text, "html.parser") 
    papers = soup.find_all('div', class_ = 'vol_cont')
    for paper in tqdm(papers):
        title = paper.find('div', class_ = 'content1').find('a').text.strip().replace('\n', '').replace(':', '').replace('/', '').replace('*', '')
        filename = os.path.join(folder_location, title + '.pdf')
        with open(filename, 'wb') as f:
            f.write(requests.get(paper.find_all('div', class_ = 'lin')[1].find('a')['href']).content)

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [11:58<00:00, 42.29s/it]


In [26]:
folder_location = r'D:\Ani\Thesis\sciencePG\Data Mining and Knowledge Discovery'
for paper in tqdm(papers):
    title = paper.find('div', class_ = 'content1').find('a').text.strip().replace('\n', '').replace(':', '').replace('/', '')
    filename = os.path.join(folder_location, title + '.pdf')
    with open(filename, 'wb') as f:
        f.write(requests.get(paper.find_all('div', class_ = 'lin')[1].find('a')['href']).content)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:20<00:00,  5.15s/it]
